In [39]:
import openai
openai.api_key = "OEPN_AI_API"




## Prompts

In [40]:
import prompts.product_predict as P

# 生成possible steps: 给反应物、生成物，生成让GPT解释possible steps的one-shot prompt
def get_generate_steps_prompt(rr, product):
    prompt = '''You are an expert chemist. Given the reactants SMILES, your task is to predict the main product SMILES using your experienced chemical Reaction Prediction knowledge. Please strictly follow the format, no other information can be provided. The input contains the reactants and reagents which are split by '.'. The product smiles must be valid and chemically reasonable. 
Reactants+Reagents: [Li]c1ccccc1.CC(=O)C(C)C.CCOCC 
products: CC(C)C([O-])c1ccccc1.[Li+] 
Possible Steps:  First, this is a remove bond step, taking the pair of electrons between the Li and C atoms and moving them to the C atom, giving rise to [Li+].  Then, a bond is added when electrons are moved from the C atom in reactant 1 to a C atom in reactant 2. Then, A pair of electrons are removed between the C and O atoms and moved to the O atom, giving rise to the one of the products CC(C)C([O-])c1ccccc1. So, the products are CC(C)C([O-])c1ccccc1.[Li+]
Reactants+Reagents: {input1}
products: {input2}
Possible Steps:
'''
    return prompt.format(input1 = rr, input2= product)
    # return P.generate_steps_one_shot.format(input1 = rr, input2= product)

# 生成最终的prompt
def create_CoT_prompt(input_text, examples):
    '''
    input_text: reactants+reagents
    examples: list of (reactants+reagents, possible steps, products); i.e. list of a tuple
    '''
    prompt = "You are an expert chemist. Please do reaction prediction and think the reaction mechanism step by step. Strictly follow the format, \
no other information can be provided, you should output two things, the possible steps and the products \n \
The input contains the reactants and reagents which are splitted by . \n \
the product smiles must be valid and chemically reasonable. \n"
    
    for example in examples:
        prompt += f"Reactants+Reagents: {example[0]}\nPossible Steps: {example[1]}\nProducts: {example[2]}\n"
    prompt += f"Reactants+Reagents: {input_text}\nPossible Steps: "
    return prompt

In [7]:
def get_ICL_prompts_reaction_type(input_text, examples):
    '''
    input_text: reactants+reagents
    examples: list of (reactants+reagents, reaction_type, products); i.e. list of a tuple
    '''

    prompt = "You are an expert chemist. Please predict the reaction type and possible products of a reaction. Strictly follow the format, \
no other information can be provided, you should output two things, the reaction type and the products \n \
The input contains the reactants and reagents which are splitted by . \n \
The product smiles must be valid and chemically reasonable. \n"

    for example in examples:
        prompt += f"Reactants+Reagents: {example[0]}\nReaction Type: {example[1]}\nProducts: {example[2]}\n"
    prompt += f"Reactants+Reagents: {input_text}\nReaction Type: "
    return prompt

In [11]:
def get_ICL_prompts_reaction_type_1(input_text, examples):
    '''
    input_text: reactants+reagents
    examples: list of (reactants+reagents, reaction_type, products); i.e. list of a tuple
    '''

    prompt = "You are an expert chemist. Please predict the reaction type and possible products of a reaction. Strictly follow the format, \
no other information can be provided, you should output two things, the reaction type and the products \n \
The input contains the reactants and reagents which are splitted by . \n \
The product smiles must be valid and chemically reasonable. Now you will be given a few examples and the final question\n"

    for example in examples:
        prompt += f"Reactants+Reagents: {example[0]}\nReaction Type: {example[1]}\nProducts: {example[2]}\n"
    prompt += f"Reactants+Reagents: {input_text}\nReaction Type: "
    return prompt

In [33]:
def get_ICL_prompts_reaction_type_desciption(input_text, examples):
    '''
    input_text: reactants+reagents
    examples: list of (reactants+reagents, reaction_type, products); i.e. list of a tuple
    '''

    prompt = "Now, you are playing as an expert chemist. Please predict the reaction type and possible products of a reaction. Strictly follow the format and no other information can be provided. \
You should output two things, the reaction type and the products. The following are description of 10 reaction type along with the possible change between SMILES of reactants and products in that reaction.\n \
    Heteroatom Alkylation and Arylation: Adding alkyl/aryl groups to heteroatoms. SMILES: Addition of groups to heteroatoms.\n \
Acylation and Related Processes: Adding acyl/carbonyl groups. SMILES: Addition of acyl/carbonyl groups.\n \
C-C Bond Formation: Creating carbon-carbon bonds. SMILES: Formation of new bonds between carbons.\n \
Heterocycle Formation: Creating heterocyclic rings. SMILES: Addition of heterocyclic rings.\n \
Protections: Temporarily modifying functional groups. SMILES: Addition of protecting groups.\n \
Deprotections: Reversing protection reactions. SMILES: Removal of protecting groups.\n \
Reductions: Decreasing oxidation state. SMILES: Modification of relevant atoms/functional groups.\n \
Oxidations: Increasing oxidation state. SMILES: Modification of relevant atoms/functional groups.\n \
Functional Group Interconversion: Converting one functional group to another. SMILES: Replacement of functional groups.\n \
Functional Group Addition (FGA): Adding new functional groups. SMILES: Addition of new functional groups.\n \
   The input contains the reactants and reagents which are splitted by . \n \
The product smiles must be valid and chemically reasonable. Now you will be given a few examples and the final question\n"

    for example in examples:
        prompt += f"Reactants+Reagents: {example[0]}\nReaction Type: {example[1]}\nProducts: {example[2]}\n"
    prompt += f"Reactants+Reagents: {input_text}\nReaction Type: "
    return prompt

In [143]:
def get_ICL_prompts(input_text, examples):
    '''
    input_text: reactants+reagents
    examples: list of (reactants+reagents, possible steps, products); i.e. list of a tuple
    '''
    prompt = "You are an expert chemist. Please do reaction prediction. Strictly follow the format, \
no other information can be provided, you should only output the products based on the examples. \n \
The input contains the reactants and reagents which are splitted by . \n \
the product smiles must be valid and chemically reasonable. \n"
    
    for example in examples:
        prompt += f"Reactants+Reagents: {example[0]}\nProducts: {example[2]}\n"
    prompt += f"Reactants+Reagents: {input_text}\nProducts: "
    return prompt

## 一、Generating Possible Steps for a target question

### read data

In [10]:
pip install rdkit

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install steamship

Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
from rdkit import Chem
from tqdm import tqdm
import os
import openai
import datetime
import numpy as np
import time

def molToCanonical(smiles):
    mol = Chem.MolFromSmiles(smiles)
    canonical_smiles = Chem.MolToSmiles(mol)
    return canonical_smiles

from steamship import Steamship

# file = 'data/reaction_prediction/uspto/uspto_mixed.pickle'
# df = pd.read_pickle(file)
# df['reactants_smiles'] = df['reactants_mol'].apply(lambda x: Chem.MolToSmiles(x))
# df['products_smiles'] = df['products_mol'].apply(lambda x: Chem.MolToSmiles(x))
# train = df[df['set'] == 'train']
# valid = df[df['set'] == 'valid']
# test = df[df['set'] == 'test']

# test = test.sample(500, random_state=42)
# valid = valid.sample(30, random_state=42)

The original dataset USPTO-50 k set provides a reaction type label for every reaction. In total, ten reaction classes ranging from protection/deprotection, to carbon–carbon bond and heterocycle formation present the most common reactions in organic synthesis.

In [256]:
# read uspto_50
import pandas as pd
from rdkit import Chem
from tqdm import tqdm
import os
import openai
import datetime
import numpy as np
import time

file = 'data/reaction_prediction/uspto/uspto_50.pickle'
df = pd.read_pickle(file)
df['reactants_smiles'] = df['reactants_mol'].apply(lambda x: Chem.MolToSmiles(x))
df['products_smiles'] = df['products_mol'].apply(lambda x: Chem.MolToSmiles(x))
train = df[df['set'] == 'train']
valid = df[df['set'] == 'valid']
test = df[df['set'] == 'test']

test = test.sample(500, random_state=42)
valid = valid.sample(30, random_state=42)

In [275]:
df[['reactants_smiles','products_smiles','Reagents_smiles','reactants_smiles_only','reaction_type']].to_csv('data/uspto_50k.csv',index=False)

In [273]:
train[['reactants_smiles','products_smiles']]

,reactants_mol,products_mol,reaction_type,set,reactants_smiles,products_smiles
0,<rdkit.Chem.rdchem.Mol object at 0x000001F2175...,<rdkit.Chem.rdchem.Mol object at 0x000001F23DA...,<RX_1>,train,CS(=O)(=O)OC[C@H]1CCC(=O)O1.Fc1ccc(Nc2ncnc3cc(...,O=C1CC[C@H](CN2CCN(CCOc3cc4ncnc(Nc5ccc(F)c(Cl)...
1,<rdkit.Chem.rdchem.Mol object at 0x000001F2175...,<rdkit.Chem.rdchem.Mol object at 0x000001F23DA...,<RX_6>,train,COC(=O)c1cc(CCCc2cc3c(=O)[nH]c(N)nc3[nH]2)cs1,Nc1nc2[nH]c(CCCc3csc(C(=O)O)c3)cc2c(=O)[nH]1
2,<rdkit.Chem.rdchem.Mol object at 0x000001F2175...,<rdkit.Chem.rdchem.Mol object at 0x000001F23DA...,<RX_9>,train,CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.FC(F)(F)c...,CC1(C)OB(c2cccc(Nc3nccc(C(F)(F)F)n3)c2)OC1(C)C
3,<rdkit.Chem.rdchem.Mol object at 0x000001F2175...,<rdkit.Chem.rdchem.Mol object at 0x000001F23DA...,<RX_6>,train,CC(C)(C)OC(=O)NCC(=O)CCC(=O)OCCCC(=O)OCc1ccccc1,CC(C)(C)OC(=O)NCC(=O)CCC(=O)OCCCC(=O)O
4,<rdkit.Chem.rdchem.Mol object at 0x000001F2175...,<rdkit.Chem.rdchem.Mol object at 0x000001F23DA...,<RX_1>,train,Fc1cc2c(Cl)ncnc2cn1.NC1CCCCCC1,Fc1cc2c(NC3CCCCCC3)ncnc2cn1
...,...,...,...,...,...,...
40024,<rdkit.Chem.rdchem.Mol object at 0x000001F2365...,<rdkit.Chem.rdchem.Mol object at 0x000001F2398...,<RX_1>,train,CCCI.CCc1c[nH]c(-c2cnc(Nc3ccc(Cl)cc3)c(Cl)c2)n1,CCCn1cc(CC)nc1-c1cnc(Nc2ccc(Cl)cc2)c(Cl)c1
40025,<rdkit.Chem.rdchem.Mol object at 0x000001F2365...,<rdkit.Chem.rdchem.Mol object at 0x000001F2398...,<RX_6>,train,COc1ccc2nccc(N3C[C@@H]4C[C@H]3CN4CCNC(=O)OCc3c...,COc1ccc2nccc(N3C[C@@H]4C[C@H]3CN4CCN)c2n1
40026,<rdkit.Chem.rdchem.Mol object at 0x000001F2365...,<rdkit.Chem.rdchem.Mol object at 0x000001F2398...,<RX_3>,train,CCOC(=O)C1(c2ccc(B3OC(C)(C)C(C)(C)O3)cc2)CC1.C...,CCOC(=O)C1(c2ccc(-c3ccc(-c4onc(C)c4NC4CCN(Cc5c...
40027,<rdkit.Chem.rdchem.Mol object at 0x000001F2365...,<rdkit.Chem.rdchem.Mol object at 0x000001F2398...,<RX_1>,train,COC(=O)c1ncc(Br)cc1F.NS(=O)(=O)c1ccncc1,COC(=O)c1ncc(NS(=O)(=O)c2ccncc2)cc1F


In [257]:
reaction_type_mapping = {
    '<RX_1>': 'heteroatom alkylation and arylation',
    '<RX_2>': 'acylation and related processes',
    '<RX_3>': 'C-C bond formation',
    '<RX_4>': 'heterocycle formation',
    '<RX_5>': 'protections',
    '<RX_6>': 'deprotections',
    '<RX_7>': 'reductions',
    '<RX_8>': 'oxidations',
    '<RX_9>': 'functional group interconversion',
    '<RX_10>': 'functional group addition (FGA)',
}

# 把df['reaction_type']里面的'<RX_1>'这种东西换成对应的reaction_type
def replace_reaction_type(x):
    for key, value in reaction_type_mapping.items():
        x = x.replace(key, value)
    return x

df['reaction_type'] = df['reaction_type'].apply(lambda x: replace_reaction_type(x))
df

,reactants_mol,products_mol,reaction_type,set,reactants_smiles,products_smiles
0,<rdkit.Chem.rdchem.Mol object at 0x000001F2175...,<rdkit.Chem.rdchem.Mol object at 0x000001F23DA...,heteroatom alkylation and arylation,train,CS(=O)(=O)OC[C@H]1CCC(=O)O1.Fc1ccc(Nc2ncnc3cc(...,O=C1CC[C@H](CN2CCN(CCOc3cc4ncnc(Nc5ccc(F)c(Cl)...
1,<rdkit.Chem.rdchem.Mol object at 0x000001F2175...,<rdkit.Chem.rdchem.Mol object at 0x000001F23DA...,deprotections,train,COC(=O)c1cc(CCCc2cc3c(=O)[nH]c(N)nc3[nH]2)cs1,Nc1nc2[nH]c(CCCc3csc(C(=O)O)c3)cc2c(=O)[nH]1
2,<rdkit.Chem.rdchem.Mol object at 0x000001F2175...,<rdkit.Chem.rdchem.Mol object at 0x000001F23DA...,functional group interconversion,train,CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.FC(F)(F)c...,CC1(C)OB(c2cccc(Nc3nccc(C(F)(F)F)n3)c2)OC1(C)C
3,<rdkit.Chem.rdchem.Mol object at 0x000001F2175...,<rdkit.Chem.rdchem.Mol object at 0x000001F23DA...,deprotections,train,CC(C)(C)OC(=O)NCC(=O)CCC(=O)OCCCC(=O)OCc1ccccc1,CC(C)(C)OC(=O)NCC(=O)CCC(=O)OCCCC(=O)O
4,<rdkit.Chem.rdchem.Mol object at 0x000001F2175...,<rdkit.Chem.rdchem.Mol object at 0x000001F23DA...,heteroatom alkylation and arylation,train,Fc1cc2c(Cl)ncnc2cn1.NC1CCCCCC1,Fc1cc2c(NC3CCCCCC3)ncnc2cn1
...,...,...,...,...,...,...
50032,<rdkit.Chem.rdchem.Mol object at 0x000001F23DA...,<rdkit.Chem.rdchem.Mol object at 0x000001F263C...,heteroatom alkylation and arylation,test,Cc1cc([N+](=O)[O-])ccc1O.Nc1cc(Cl)ccn1,Cc1cc([N+](=O)[O-])ccc1Oc1ccnc(N)c1
50033,<rdkit.Chem.rdchem.Mol object at 0x000001F23DA...,<rdkit.Chem.rdchem.Mol object at 0x000001F263C...,deprotections,test,COC(=O)c1[nH]c2cc(Cl)cc3c2c1C(CC(=O)OC(C)(C)C)CC3,COC(=O)c1[nH]c2cc(Cl)cc3c2c1C(CC(=O)O)CC3
50034,<rdkit.Chem.rdchem.Mol object at 0x000001F23DA...,<rdkit.Chem.rdchem.Mol object at 0x000001F263C...,reductions,test,COc1cc(C(F)(F)F)cc(SC)c1C(=O)NC1(c2ccccc2)CC(=...,COc1cc(C(F)(F)F)cc(SC)c1C(=O)NC1(c2ccccc2)CC(O...
50035,<rdkit.Chem.rdchem.Mol object at 0x000001F23DA...,<rdkit.Chem.rdchem.Mol object at 0x000001F263C...,functional group addition (FGA),test,C=C(C)Cn1nc(C)c(Br)c1-c1ccc(F)cc1.OO,Cc1nn(CC(C)CO)c(-c2ccc(F)cc2)c1Br


In [24]:
df['reaction_type'].value_counts()

reaction_type
<RX_1>     15122
<RX_2>     11913
<RX_6>      8353
<RX_3>      5639
<RX_7>      4585
<RX_9>      1834
<RX_4>       900
<RX_8>       814
<RX_5>       650
<RX_10>      227
Name: count, dtype: int64

In [23]:
# reaction_type的百分比
df['reaction_type'].value_counts(normalize=True)

reaction_type
<RX_1>     0.302216
<RX_2>     0.238084
<RX_6>     0.166936
<RX_3>     0.112697
<RX_7>     0.091632
<RX_9>     0.036653
<RX_4>     0.017987
<RX_8>     0.016268
<RX_5>     0.012990
<RX_10>    0.004537
Name: proportion, dtype: float64

In [13]:
from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Chem import DataStructs
from rdkit.Chem import rdMolDescriptors
from tqdm import tqdm

# "Scaffold fingerprint"（脚手架指纹）是一种用于描述化学分子结构的编码表示方法。它用于比较和分类化合物，通常用于虚拟筛选、药物设计和化学库分析等领域
# 脚手架指纹是通过将化合物的结构转化为一系列二进制位（或比特位）的向量来表示的。
# 这些二进制位通常编码了分子中的特定化学性质、功能基团或子结构。通过对这些位进行比较，可以量化和比较不同分子之间的相似性。
def get_scaffold_fp(x):
    mol = Chem.MolFromSmiles(x)
    scaffold = MurckoScaffold.GetScaffoldForMol(mol)
    scaffold_fp = rdMolDescriptors.GetMorganFingerprint(scaffold, 2)
    return scaffold_fp

In [14]:
# 获取脚手架指纹
train['scaffold_fp'] = train['reactants_smiles'].apply(lambda x: get_scaffold_fp(x))

[01:44:54] WARNING: not removing hydrogen atom without neighbors
[01:44:54] WARNING: not removing hydrogen atom without neighbors
[01:44:54] WARNING: not removing hydrogen atom without neighbors
[01:44:54] WARNING: not removing hydrogen atom without neighbors
[01:44:54] WARNING: not removing hydrogen atom without neighbors
[01:44:54] WARNING: not removing hydrogen atom without neighbors
[01:44:54] WARNING: not removing hydrogen atom without neighbors
[01:44:54] WARNING: not removing hydrogen atom without neighbors
[01:44:54] WARNING: not removing hydrogen atom without neighbors
[01:44:54] WARNING: not removing hydrogen atom without neighbors
[01:44:54] WARNING: not removing hydrogen atom without neighbors
[01:44:54] WARNING: not removing hydrogen atom without neighbors
[01:44:54] WARNING: not removing hydrogen atom without neighbors
[01:44:54] WARNING: not removing hydrogen atom without neighbors
[01:44:54] WARNING: not removing hydrogen atom without neighbors
[01:44:54] WARNING: not r

In [266]:
df['scaffold_fp'] = df['reactants_smiles'].apply(lambda x: get_scaffold_fp(x))

In [24]:
# 获取与目标分子(SMILES格式)具有相似骨架(scaffold fingerprint)的前n个分子(SMILES格式), 以及在数据集中的id
def top_n_scaffold_similar_molecules(target_smiles, molecule_scaffold_list, molecule_smiles_list, n=5):
    # 将目标分子的SMILES字符串转换为RDKit的分子对象
    target_mol = Chem.MolFromSmiles(target_smiles)

    # 生成目标分子的Murcko Scaffold骨架
    target_scaffold = MurckoScaffold.GetScaffoldForMol(target_mol)
    
    # 使用Morgan指纹计算目标骨架的指纹fingerprint
    target_fp = rdMolDescriptors.GetMorganFingerprint(target_scaffold, 2)

    # 存储相似性结果的列表
    similarities = []

    # 遍历分子骨架列表
    for idx, scaffold_fp in enumerate(molecule_scaffold_list):
        try:
            tanimoto_similarity = DataStructs.TanimotoSimilarity(target_fp, scaffold_fp) # 计算目标骨架指纹与当前分子骨架指纹的Tanimoto相似度
            if target_fp!=scaffold_fp:
                similarities.append((idx, tanimoto_similarity))  # 将相似度结果添加到相似性列表中
        except Exception as e:
            print(e)
            continue

    similarities.sort(key=lambda x: x[1], reverse=True) # 根据相似度进行排序（从高到低）
    top_5_similar_molecules = similarities[:n] # 获取前n个最相似的分子

    return [molecule_smiles_list[i[0]] for i in top_5_similar_molecules], [i[0] for i in top_5_similar_molecules]

获取与target reactants smiles相似的前5个reactants smiles

In [25]:
simi_0 = top_n_scaffold_similar_molecules(train.iloc[0]['reactants_smiles'], list(train['scaffold_fp']), list(train['reactants_smiles']), n=5)

In [26]:
train.iloc[0]['reactants_smiles']

'C1CCOC1.CC(C)C[Mg+].CON(C)C(=O)c1ccc(O)nc1.[Cl-]'

In [27]:
simi_0

(['Cc1nc(O)ccc1Br.OC1CCOCC1',
  'C#CCOC1CCCCO1.CCNCC.ClCCl.Nc1ccc(I)cn1.O.[Cu]I',
  'CCN(CC)CC.CN(C)c1ccncc1.CS(=O)(=O)Cl.ClCCl.OC1CCOCC1',
  'CCN(CC)CC.CCOC(C)=O.CS(C)=O.Cl.N#Cc1ncc(F)cc1F.NC1CCOCC1.O.O=C([O-])[O-].[K+].[K+]',
  'CC(=O)C1CCOCC1.CC(=O)O.CO.Nc1cc(Br)cnc1N.O=C([O-])O.[BH3-]C#N.[Na+].[Na+]'],
 [2986, 9977, 42406, 47933, 50186])

In [21]:
train.iloc[302]['products_smiles']

'CC(=O)NC(C)(C)CC(=O)O'

### read data-2

In [114]:
import pandas as pd

data = {'reactants_smiles': [], 'products_smiles': [], 'reaction_type': []}

for a in ['Buchwald-Hartwig', 'Chan-Lam', 'Hiyama', 'Kumada', 'Suzuki']:
    # 打开并读取 .source 文件
    source_file_path = f"data\\reaction_data\\data\\{a}\\raw data\\dev.source"  # 将文件路径替换为实际的文件路径

    with open(source_file_path, 'r', encoding='utf-8') as source_file:
        source_lines = source_file.readlines()

    # 打开并读取 .target 文件
    target_file_path = f"data\\reaction_data\\data\\{a}\\raw data\\dev.target"  # 将文件路径替换为实际的文件路径

    with open(target_file_path, 'r', encoding='utf-8') as target_file:
        target_lines = target_file.readlines()

    # 添加数据到DataFrame
    for source_line, target_line in zip(source_lines, target_lines):
        data['reactants_smiles'].append(source_line.strip())
        data['products_smiles'].append(target_line.strip())
        data['reaction_type'].append(a)

df_5reaction = pd.DataFrame(data)


In [282]:
df_5reaction.to_csv('data/5reaction.csv',index=False)

In [117]:
df_5reaction['scaffold_fp'] = df_5reaction['reactants_smiles'].apply(lambda x: get_scaffold_fp(x))

[02:57:44] WARNING: not removing hydrogen atom without neighbors
[02:57:46] WARNING: not removing hydrogen atom without neighbors
[02:57:46] WARNING: not removing hydrogen atom without neighbors
[02:57:46] WARNING: not removing hydrogen atom without neighbors
[02:57:46] WARNING: not removing hydrogen atom without neighbors


In [118]:
# df_5reaction划分train, valid, test
df_5reaction['set'] = 'train'
df_5reaction.loc[df_5reaction.sample(frac=0.1, random_state=42).index, 'set'] = 'test'
df_5reaction.loc[df_5reaction.sample(100, random_state=40).index, 'set'] = 'valid'

# 获得train, valid, test
train_5reaction = df_5reaction[df_5reaction['set'] == 'train']
valid_5reaction = df_5reaction[df_5reaction['set'] == 'valid']
test_5reaction = df_5reaction[df_5reaction['set'] == 'test']

### get GPT response & Evaluation

In [45]:
def generate_response_by_davinci(prompt, model_engine = 'text-davinci-003',n=1, temperature=0.5):
    time.sleep(2)
    completion = openai.Completion.create(
      engine=model_engine,
      prompt=prompt,
      temperature=temperature,
      max_tokens=256,
#       top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0, 
      n=n,
    )
#     message = completion.choices[0]['text'].strip()
    
    message = completion.choices
    message = [i['text'].strip() for i in message]
    return message

def generate_response_by_gpt35(prompt, model_engine = "gpt-3.5-turbo",n=1, temperature=0.5):
    time.sleep(2)
    completion = openai.ChatCompletion.create(
        model=model_engine, temperature=temperature, n=n, 
        messages=[{"role": "user", "content": prompt}],
    )
#     message = completion.choices[0].message.content.strip()

    message = completion.choices
    message = [i.message.content.strip() for i in message]
    return message

def generate_response_by_gpt4(prompt,n=1, temperature=0.5):
    # Create a Steamship client
    # NOTE: When developing a package, just use `self.client`
    time.sleep(2)
    client = Steamship(workspace="gpt-4-92g")

    # Create an instance of this generator
    generator = client.use_plugin('gpt-4', config={"temperature":temperature, "n": n})

    # Generate text
    task = generator.generate(text=prompt)
    # Wait for completion of the task.
    task.wait()
    # Print the output
#     message = task.output.blocks[0].text.strip()
    message = task.output.blocks
    message = [i.text.strip() for i in message]
    return message

In [130]:
np.random.seed(42)

def predict_test(target, model = "gpt35", cot_type = "simi", examples_n=10, samples_n=5):
    assert cot_type in ["simi", "random"]
    if cot_type == "simi":
        # get similar molecules
        simi = top_n_scaffold_similar_molecules(target,list(train['scaffold_fp']), list(train['reactants_smiles']), n=examples_n)
        examples = []
        for i in range(examples_n):
            prompt_gene = get_generate_steps_prompt(simi[0][i], train.iloc[simi[1][i]]['products_smiles'])
            if model == "gpt35":
                possible_steps = generate_response_by_gpt35(prompt_gene)[0]
            elif model == "gpt4":
                possible_steps = generate_response_by_gpt4(prompt_gene)[0]
            # print(possible_steps)
            examples.append((simi[0][i], possible_steps, train.iloc[simi[1][i]]['products_smiles']))

    elif cot_type == "random":
        examples = []
        random_idx = np.random.randint(0, len(train), examples_n)
        for i in random_idx:
            prompt_gene = get_generate_steps_prompt(train.iloc[i]['reactants_smiles'], train.iloc[i]['products_smiles'])
            if model == "gpt35":
                possible_steps = generate_response_by_gpt35(prompt_gene)[0]
            elif model == "gpt4":
                possible_steps = generate_response_by_gpt4(prompt_gene)[0]
            # print(possible_steps)
            examples.append((train.iloc[i]['reactants_smiles'], possible_steps, train.iloc[i]['products_smiles']))

    final_prompt = create_CoT_prompt(target, examples)
    if model == "gpt35":
        response = generate_response_by_gpt35(final_prompt, n=samples_n)
    elif model == "gpt4":
        response = generate_response_by_gpt4(final_prompt, n=samples_n)
    # print('------------------------------------------')
    # print(final_prompt)
    # print('------------------------------------------')
    # print(response)
    products_predict_list = [r.split('\n')[-1].split(' ')[-1] for r in response]
    return response, products_predict_list

In [131]:
def experiment_eval(test_number = 10, model = "gpt35", cot_type = "simi", examples_n=3, samples_n=5):
    # query
    correct = 0
    test_ = test.sample(test_number, random_state=42) # TODO
    for idx, row in tqdm(test_.iterrows()):
        reactant = row['reactants_smiles']
        product = row['products_smiles']
        # index = row['index']
        
        print('begin!',idx)
        response, products_predict_list = predict_test(reactant, model = model, cot_type = cot_type, examples_n=examples_n, samples_n=samples_n)
        correct_flag = False
        for pred in products_predict_list:
            try:
                mol = Chem.MolFromSmiles(pred)
                pred = Chem.MolToSmiles(mol)
            except Exception as e:
                continue

            pred_list = pred.split(".")
            if product in pred_list:
                correct_flag = True
                correct += 1
                break

    top_n_acc = correct/len(test_)
    return top_n_acc


In [133]:
top_n_acc = experiment_eval(test_number = 20, model = "gpt35", cot_type = "simi", examples_n=3, samples_n=5)
print(top_n_acc)

0it [00:00, ?it/s]

begin! 442911


[04:20:06] Can't kekulize mol.  Unkekulized atoms: 21 22 23 24 26
[04:20:06] SMILES Parse Error: syntax error while parsing: CCOC(C)=O.COC(=O)C1CC(OS(C)(=O)=O)CN1Cc1cccnc1.[Li+].[Cl-].
[04:20:06] SMILES Parse Error: Failed parsing SMILES 'CCOC(C)=O.COC(=O)C1CC(OS(C)(=O)=O)CN1Cc1cccnc1.[Li+].[Cl-].' for input: 'CCOC(C)=O.COC(=O)C1CC(OS(C)(=O)=O)CN1Cc1cccnc1.[Li+].[Cl-].'
[04:20:06] Can't kekulize mol.  Unkekulized atoms: 21 22 23 24 26
[04:20:06] SMILES Parse Error: syntax error while parsing: CCOC(C)=O.COC(=O)C1CC(OS(C)(=O)=O)CN1Cc1cccnc1.[Cl-].[Li+].
[04:20:06] SMILES Parse Error: Failed parsing SMILES 'CCOC(C)=O.COC(=O)C1CC(OS(C)(=O)=O)CN1Cc1cccnc1.[Cl-].[Li+].' for input: 'CCOC(C)=O.COC(=O)C1CC(OS(C)(=O)=O)CN1Cc1cccnc1.[Cl-].[Li+].'
1it [00:25, 25.14s/it]

------------------------------------------
Please do reaction prediction and think the reaction mechanism step by step. There are some rules needed to follow: 
 1. Strictly follow the format, no other information can be provided, you should only reply with SMILES string notations to represent the product. 
 2. The input contains the reactants and reagents which are splitted by . 
 3. the product smiles must be valid and chemically reasonable. 
Reactants+Reagents: CCOC(=O)CC1CCCCN1Cc1cccnc1OC.CO.Cl.[Na+].[OH-]
Possible Steps: First, a bond is added between the C atom in reactant 1 and the C atom in reactant 2, resulting in CCOC(=O)CC1CCCCN1Cc1cccnc1OC.CO. Then, a bond is added between the Cl atom and the Na atom, giving rise to COc1ncccc1CN1CCCCC1CC(=O)O. Therefore, the product is COc1ncccc1CN1CCCCC1CC(=O)O.
Products: COc1ncccc1CN1CCCCC1CC(=O)O
Reactants+Reagents: C=CC(=O)OC(C)(C)C.CC(=O)[O-].CC(=O)[O-].Nc1ncc(Br)cc1CN1CCCCC1.[Pd+2]
Possible Steps: First, a bond is added between the C a

[04:20:33] SMILES Parse Error: syntax error while parsing: CCC(NCC(C)O)C(=O)OCC(C)C.
[04:20:33] SMILES Parse Error: Failed parsing SMILES 'CCC(NCC(C)O)C(=O)OCC(C)C.' for input: 'CCC(NCC(C)O)C(=O)OCC(C)C.'
[04:20:33] SMILES Parse Error: syntax error while parsing: CCC(N)C(=O)OCC(C)Cc1ccc(Cl)cc1Cl.
[04:20:33] SMILES Parse Error: Failed parsing SMILES 'CCC(N)C(=O)OCC(C)Cc1ccc(Cl)cc1Cl.' for input: 'CCC(N)C(=O)OCC(C)Cc1ccc(Cl)cc1Cl.'
[04:20:33] SMILES Parse Error: syntax error while parsing: CCC(NCC(Oc1ccc(Cl)cc1)C)C(=O)OCC(C)C.
[04:20:33] SMILES Parse Error: Failed parsing SMILES 'CCC(NCC(Oc1ccc(Cl)cc1)C)C(=O)OCC(C)C.' for input: 'CCC(NCC(Oc1ccc(Cl)cc1)C)C(=O)OCC(C)C.'
2it [00:51, 25.72s/it]

------------------------------------------
Please do reaction prediction and think the reaction mechanism step by step. There are some rules needed to follow: 
 1. Strictly follow the format, no other information can be provided, you should only reply with SMILES string notations to represent the product. 
 2. The input contains the reactants and reagents which are splitted by . 
 3. the product smiles must be valid and chemically reasonable. 
Reactants+Reagents: CC(C)(C)OCC1CO1.COc1ccc(CC(C)(C)N)cc1
Possible Steps: First, a bond is added between the C atom in reactant 1 and the N atom in reactant 2, resulting in CC(C)(C)NCC(O)COC(C)(C)C. Then, a bond is added between the O atom in reactant 1 and the C atom in reactant 2, resulting in COc1ccc(CC(C)(C)NCC(O)COC(C)(C)C)cc1. Therefore, the product is COc1ccc(CC(C)(C)NCC(O)COC(C)(C)C)cc1.
Products: COc1ccc(CC(C)(C)NCC(O)COC(C)(C)C)cc1
Reactants+Reagents: CC(C)=O.ClCC1(CCl)CO1.O=C([O-])[O-].Oc1ccc(Cl)cc1.[K+].[K+]
Possible Steps: First, the

#### draft

In [79]:
train.iloc[0]['reactants_smiles']

'C1CCOC1.CC(C)C[Mg+].CON(C)C(=O)c1ccc(O)nc1.[Cl-]'

In [80]:
train.iloc[0]['products_smiles']

'CC(C)CC(=O)c1ccc(O)nc1'

In [115]:
response, products_predict_list = predict_test("C1CCOC1.CC(C)C[Mg+].CON(C)C(=O)c1ccc(O)nc1.[Cl-]")

First, a bond is added when electrons are moved from the O atom in reactant 2 to the C atom in reactant 1, giving rise to Cc1nc(OC2CCOCC2)ccc1Br. So, the product is Cc1nc(OC2CCOCC2)ccc1Br.
First, a bond is added between the C and O atoms in reactant 1 and the C atom in reactant 2, giving rise to C#CCOC1CCCCO1. Then, a bond is added between the N atom in reactant 4 and the C atom in the product, giving rise to Nc1ccc(C#CCOC2CCCCO2)cn1. So, the product is Nc1ccc(C#CCOC2CCCCO2)cn1.
First, a bond is added when electrons are moved from the C atom in reactant 1 to a C atom in reactant 2. Then, a bond is formed between the S atom in reactant 3 and the O atom in reactant 4, resulting in the product CS(=O)(=O)OC1CCOCC1. So, the product is CS(=O)(=O)OC1CCOCC1.
First, the Cl atom in reactant 4 is replaced by the N atom in reactant 1, giving rise to N#Cc1ncc(F)cc1Cl. Then, the N atom in reactant 1 is replaced by the NC1CCOCC1 group in reactant 4, giving rise to N#Cc1ncc(F)cc1NC1CCOCC1. So, the pro

In [116]:
products_predict_list

['CC(C)C[Mg+].CON(C)c1cccnc1',
 'CC(C)[Mg+].CON(C)C(=O)c1ccc(O)nc1.NC1CCOC1=O',
 'CC(C)CC1CCOC1.Cl',
 'CC(C)CC1CCOC1.[Mg+].[Cl-]',
 'CC(C)C[Mg+](OCON(C)C(=O)c1ccc(O)nc1)C1CCOC1']

In [96]:
# question: C1CCOC1.CC(C)C[Mg+].CON(C)C(=O)c1ccc(O)nc1.[Cl-]
# true answer: CC(C)CC(=O)c1ccc(O)nc1
for r in res:
    print(r.split('\n')[-1].split(' ')[-1])

CON(C)C(=O)c1ccc(O)nc1
CC1CCOC1.
C1CCOC1
C1CCOC1.CC(C)C[Mg+]
CC(C)C[Mg+].CON(C)C(=O)c1ccc(OC2CCOCC2)nc1


In [99]:
[r.split('\n')[-1].split(' ')[-1] for r in res]

['CON(C)C(=O)c1ccc(O)nc1',
 'CC1CCOC1.',
 'C1CCOC1',
 'C1CCOC1.CC(C)C[Mg+]',
 'CC(C)C[Mg+].CON(C)C(=O)c1ccc(OC2CCOCC2)nc1']

## 二、Add Reaction Type

In [1]:
import openai
openai.api_key = "OEPN_AI_API"

In [2]:
import pandas as pd
from rdkit import Chem
from tqdm import tqdm
import os
import openai
import datetime
import numpy as np
import time

from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Chem import DataStructs
from rdkit.Chem import rdMolDescriptors
from tqdm import tqdm


def get_scaffold_fp(x):
    mol = Chem.MolFromSmiles(x)
    scaffold = MurckoScaffold.GetScaffoldForMol(mol)
    scaffold_fp = rdMolDescriptors.GetMorganFingerprint(scaffold, 2)
    return scaffold_fp

def top_n_scaffold_similar_molecules(target_smiles, molecule_scaffold_list, molecule_smiles_list, n=5):
    '''
    获取与目标分子(SMILES格式)具有相似骨架(scaffold fingerprint)的前n个分子(SMILES格式), 以及在数据集中的id
    '''

    target_mol = Chem.MolFromSmiles(target_smiles) # 将目标分子的SMILES字符串转换为RDKit的分子对象
    target_scaffold = MurckoScaffold.GetScaffoldForMol(target_mol) # 生成目标分子的Murcko Scaffold骨架
    target_fp = rdMolDescriptors.GetMorganFingerprint(target_scaffold, 2) # 使用Morgan指纹计算目标骨架的指纹fingerprint
    
    similarities = [] # 存储相似性结果的列表

    # 遍历分子骨架列表
    for idx, scaffold_fp in enumerate(molecule_scaffold_list):
        try:
            tanimoto_similarity = DataStructs.TanimotoSimilarity(target_fp, scaffold_fp) # 计算目标骨架指纹与当前分子骨架指纹的Tanimoto相似度
            if target_fp!=scaffold_fp:
                similarities.append((idx, tanimoto_similarity))  # 将相似度结果添加到相似性列表中
        except Exception as e:
            print(e)
            continue

    similarities.sort(key=lambda x: x[1], reverse=True) # 根据相似度进行排序（从高到低）
    top_5_similar_molecules = similarities[:n] # 获取前n个最相似的分子

    return [molecule_smiles_list[i[0]] for i in top_5_similar_molecules], [i[0] for i in top_5_similar_molecules]


def process_dataset(dataset_name):
    assert dataset_name in ['uspto_50', 'uspto_mixed']

    # Read dataset
    file = f'data/reaction_prediction/uspto/{dataset_name}.pickle'
    df = pd.read_pickle(file)
    
    # Generate SMILES strings for reactants and products
    df['reactants_smiles'] = df['reactants_mol'].apply(lambda x: Chem.MolToSmiles(x))
    df['products_smiles'] = df['products_mol'].apply(lambda x: Chem.MolToSmiles(x))
    
    # for uspto_50
    if dataset_name == 'uspto_50':
        # Mapping for replacing reaction types
        reaction_type_mapping = {
            '<RX_1>': 'heteroatom alkylation and arylation',
            '<RX_2>': 'acylation and related processes',
            '<RX_3>': 'C-C bond formation',
            '<RX_4>': 'heterocycle formation',
            '<RX_5>': 'protections',
            '<RX_6>': 'deprotections',
            '<RX_7>': 'reductions',
            '<RX_8>': 'oxidations',
            '<RX_9>': 'functional group interconversion',
            '<RX_10>': 'functional group addition (FGA)',
        }
        
        # Replace reaction types in the 'reaction_type' column
        def replace_reaction_type(x):
            for key, value in reaction_type_mapping.items():
                x = x.replace(key, value)
            return x
        
        df['reaction_type'] = df['reaction_type'].apply(lambda x: replace_reaction_type(x))



    # Split dataset into train, valid, and test sets
    train = df[df['set'] == 'train']
    valid = df[df['set'] == 'valid']
    test = df[df['set'] == 'test']
    
    # 获取脚手架指纹
    train['scaffold_fp'] = train['reactants_smiles'].apply(lambda x: get_scaffold_fp(x))
    
    # Sample a subset of test and valid sets
    test = test.sample(500, random_state=42)
    valid = valid.sample(30, random_state=42)

    return df, train, valid, test



In [3]:

dataset_name = 'uspto_50'
uspto_50, train, valid, test = process_dataset(dataset_name)

C:\Users\22100\AppData\Local\Temp\ipykernel_112812\3352191439.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['scaffold_fp'] = train['reactants_smiles'].apply(lambda x: get_scaffold_fp(x))


In [47]:
train.head(3)

,reactants_mol,products_mol,reaction_type,set,reactants_smiles,products_smiles,scaffold_fp
0,<rdkit.Chem.rdchem.Mol object at 0x0000024DA1E...,<rdkit.Chem.rdchem.Mol object at 0x0000024C2D9...,heteroatom alkylation and arylation,train,CS(=O)(=O)OC[C@H]1CCC(=O)O1.Fc1ccc(Nc2ncnc3cc(...,O=C1CC[C@H](CN2CCN(CCOc3cc4ncnc(Nc5ccc(F)c(Cl)...,<rdkit.DataStructs.cDataStructs.UIntSparseIntV...
1,<rdkit.Chem.rdchem.Mol object at 0x0000024D40E...,<rdkit.Chem.rdchem.Mol object at 0x0000024C2D9...,deprotections,train,COC(=O)c1cc(CCCc2cc3c(=O)[nH]c(N)nc3[nH]2)cs1,Nc1nc2[nH]c(CCCc3csc(C(=O)O)c3)cc2c(=O)[nH]1,<rdkit.DataStructs.cDataStructs.UIntSparseIntV...
2,<rdkit.Chem.rdchem.Mol object at 0x0000024C2E9...,<rdkit.Chem.rdchem.Mol object at 0x0000024C2D9...,functional group interconversion,train,CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.FC(F)(F)c...,CC1(C)OB(c2cccc(Nc3nccc(C(F)(F)F)n3)c2)OC1(C)C,<rdkit.DataStructs.cDataStructs.UIntSparseIntV...


### 1.一次性（只query一次GPT）的Majority Vote

In [44]:
uspto_50['reactants_smiles'].iloc[0]

'CS(=O)(=O)OC[C@H]1CCC(=O)O1.Fc1ccc(Nc2ncnc3cc(OCCN4CCNCC4)c(OC4CCCC4)cc23)cc1Cl'

In [53]:
uspto_50['products_smiles'].iloc[0]

'O=C1CC[C@H](CN2CCN(CCOc3cc4ncnc(Nc5ccc(F)c(Cl)c5)c4cc3OC3CCCC3)CC2)O1'

In [60]:
uspto_50['reaction_type'].iloc[0]

'heteroatom alkylation and arylation'

In [64]:
def generate_response_by_davinci(prompt, model_engine = 'text-davinci-003',n=1, temperature=0.5):
    time.sleep(2)
    completion = openai.Completion.create(
      engine=model_engine,
      prompt=prompt,
      temperature=temperature,
      max_tokens=256,
#       top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0, 
      n=n,
    )
#     message = completion.choices[0]['text'].strip()
    
    message = completion.choices
    message = [i['text'].strip() for i in message]
    return message

def generate_response_by_gpt35(prompt, model_engine = "gpt-3.5-turbo",n=1, temperature=0.5):
    time.sleep(2)
    completion = openai.ChatCompletion.create(
        model=model_engine, temperature=temperature, n=n, 
        messages=[{"role": "user", "content": prompt}],
    )
#     message = completion.choices[0].message.content.strip()

    message = completion.choices
    message = [i.message.content.strip() for i in message]
    return message

def generate_response_by_gpt4(prompt,n=1, temperature=0.5):
    # Create a Steamship client
    # NOTE: When developing a package, just use `self.client`
    time.sleep(2)
    client = Steamship(workspace="gpt-4-92g")

    # Create an instance of this generator
    generator = client.use_plugin('gpt-4', config={"temperature":temperature, "n": n})

    # Generate text
    task = generator.generate(text=prompt)
    # Wait for completion of the task.
    task.wait()
    # Print the output
#     message = task.output.blocks[0].text.strip()
    message = task.output.blocks
    message = [i.text.strip() for i in message]
    return message

##### predict with no reaction type

In [65]:
def create_prompt_final(input_text, examples):
    prompt = "You are an expert chemist. Given the reactants SMILES, your task is to predict the main product SMILES using your experienced chemical Reaction Prediction knowledge. \n\
Please strictly follow the format, no other information can be provided. You should only reply with SMILES string notations to represent the product. The input contains the reactants and reagents which are split by '.'. The product smiles must be valid and chemically reasonable. \n"
    
    for example in examples:
        prompt += f"Reactants+Reagents: {example[0]}\nProducts: {example[1]}\n"
    prompt += f"Reactants+Reagents: {input_text}\nProducts:"
    return prompt

In [ ]:
params = [
    # ('Scaffold_SIM', 20, 'SMILES', 'gpt4'),
    ('Scaffold_SIM', 20, 'SMILES', 'davinci003'),
    ('Scaffold_SIM', 20, 'SMILES', 'gpt35'),
    
    # ('Scaffold_SIM', 5, 'SMILES', 'gpt4'),
    # ('Fixed_ICL', 20, 'SMILES', 'gpt4'),
]

In [ ]:
detail_save_folder = 'log/'

for sample_method, sample_num, name, model in params:
    
    performance_results = []
    details_results = []

    detail_predict_file = detail_save_folder + 'valid_{}_{}_{}_{}.csv'.format(model, name, sample_num, sample_method)
    log_file = detail_save_folder + 'valid_{}_{}_{}_{}.log'.format(model, name, sample_num, sample_method)
    performance_file = detail_save_folder + 'valid_performance_{}_{}_{}_{}.csv'.format(model, name, sample_num, sample_method)
    print(detail_predict_file)
            
            
    if os.path.exists(performance_file):
        print("{} exits, continue to next params".format(performance_file))
        continue


    # append new date
    # Get the current date and time
    now = datetime.datetime.now()
    # Convert the date and time to a string
    date_time_str = now.strftime("%Y-%m-%d %H:%M:%S")
    with open(log_file, "a") as file:
        file.write("=" * 30 + date_time_str + "=" * 30 + "\n")

    # init var
    predicted_products = []
    
    
    # restore previous data and skip 
    previous_index = []
    if os.path.exists(detail_predict_file):
        previous = pd.read_csv(detail_predict_file)
        previous_index = list(previous['index'])
        
        # previous data => details_results 
        details_results = previous.values.tolist()
        predicted_products = previous[['pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5']].values.tolist()

    # query
    for idx, row in tqdm(test.iterrows()):
        reactant = row['reactants_smiles']
        product = row['products_smiles']
        index = row['index']
        
        if index in previous_index:
            continue

        # ICL examples
        if sample_method == 'Fixed_ICL':
            chunk = train.sample(sample_num, random_state=42)
        elif sample_method == 'Scaffold_SIM':
            sim = top_n_scaffold_similar_molecules(reactant, list(train['scaffold_fp']), list(train['reactants_smiles']), n=sample_num)
            chunk = train[train['reactants_smiles'].isin(sim)]
        examples = list(zip(chunk["reactants_smiles"].values, chunk["products_smiles"].values))

        # build prompt and save
        prompt = create_prompt_final(reactant, examples)
        with open(log_file, "a") as file:
            file.write(prompt + "\n")
            file.write("=" * 50 + "\n")
        
        print(prompt)

        try:
            # different model 
            if model == 'davinci003':
                predicted_product = generate_response_by_davinci(prompt)
            elif model == 'gpt35':
                predicted_product = generate_response_by_gpt35(prompt)
            elif model == 'gpt4':
                predicted_product = generate_response_by_gpt4(prompt)
        except Exception as e:
            print(e)
            if "Please reduce your prompt; or completion length." in str(e):
                
                try:
                
                    print("using {} icl examples".format(sample_num - 5))

                    # ICL examples
                    if sample_method == 'Fixed_ICL':
                        chunk = train.sample(sample_num-5, random_state=42)
                    elif sample_method == 'Scaffold_SIM':
                        sim = top_n_scaffold_similar_molecules(reactant, list(train['scaffold_fp']), list(train['reactants_smiles']), n=sample_num-5)
                        chunk = train[train['reactants_smiles'].isin(sim)]
                    examples = list(zip(chunk["reactants_smiles"].values, chunk["products_smiles"].values))

                    # build prompt and save
                    prompt = create_prompt_final(reactant, examples)
                    with open(log_file, "a") as file:
                        file.write(prompt + "\n")
                        file.write("=" * 50 + "\n")

                    # different model 
                    if model == 'davinci003':
                        predicted_product = generate_response_by_davinci(prompt)
                    elif model == 'gpt35':
                        predicted_product = generate_response_by_gpt35(prompt)
                    elif model == 'gpt4':
                        predicted_product = generate_response_by_gpt4(prompt)
                except Exception as e:
                    print(e)
                    if "Please reduce your prompt; or completion length." in str(e):
                        print("using {} icl examples".format(sample_num - 10))

                        # ICL examples
                        if sample_method == 'Fixed_ICL':
                            chunk = train.sample(sample_num-10, random_state=42)
                        elif sample_method == 'Scaffold_SIM':
                            sim = top_n_scaffold_similar_molecules(reactant, list(train['scaffold_fp']), list(train['reactants_smiles']), n=sample_num-10)
                            chunk = train[train['reactants_smiles'].isin(sim)]
                        examples = list(zip(chunk["reactants_smiles"].values, chunk["products_smiles"].values))

                        # build prompt and save
                        prompt = create_prompt_final(reactant, examples)
                        with open(log_file, "a") as file:
                            file.write(prompt + "\n")
                            file.write("=" * 50 + "\n")

                        # different model 
                        if model == 'davinci003':
                            predicted_product = generate_response_by_davinci(prompt)
                        elif model == 'gpt35':
                            predicted_product = generate_response_by_gpt35(prompt)
                        elif model == 'gpt4':
                            predicted_product = generate_response_by_gpt4(prompt)
                    else:
                        raise RuntimeError("err")
                
            else:
                raise RuntimeError("err")

        predicted_products.append(predicted_product)
        details_results.append([index, reactant] + [product] + predicted_product)
        
        details_df = pd.DataFrame(details_results, columns=['index', 'reactant', 'product', 'pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5'])
        details_df.to_csv(detail_predict_file, index=False)
        
        # evaluate
        if len(predicted_products) % 50 == 0:
            # evaluate
            acc_list = []
            for repeat in range(5):
                tpredicted_products = [i[repeat] for i in predicted_products]
                correct = 0
                all_sample_num = len(predicted_products)
                for idx, gt in enumerate(list(test.head(all_sample_num)['products_smiles'])):
                    pred = tpredicted_products[idx]
                    try:
                        mol = Chem.MolFromSmiles(pred)
                        pred = Chem.MolToSmiles(mol)
                    except Exception as e:
                        continue

                    pred_list = pred.split(".")
                    if gt in pred_list:
                        correct += 1
                acc = correct / all_sample_num
                acc_list.append(acc)
            print(np.mean(acc_list))
        

    # evaluate
    acc_list = []
    for repeat in range(5):
        tpredicted_products = [i[repeat] for i in predicted_products]
        correct = 0
        all_sample_num = len(test)
        for idx, gt in enumerate(list(test['products_smiles'])):
            pred = tpredicted_products[idx]
            try:
                mol = Chem.MolFromSmiles(pred)
                pred = Chem.MolToSmiles(mol)
            except Exception as e:
                continue

            pred_list = pred.split(".")
            if gt in pred_list:
                correct += 1
        acc = correct / all_sample_num
        acc_list.append(acc)

    # save to file
    performance_results.append([model, name, sample_num, sample_method, np.mean(acc_list)] + acc_list)
    print(performance_results)

    # performance save based on the task
    tem = pd.DataFrame(performance_results, columns=['model', 'name', 'sample_num', 'sample_method', 'avg_metric'] + ['metric_{}'.format(i) for i in range(5)])
    tem.to_csv(performance_file, index=False)

##### predict_with_reaction_type

In [140]:
np.random.seed(42)

def predict_with_reaction_type(target, model = "gpt35", cot_type = "simi", examples_n=20, samples_n=5, temperature=0.5, reaction_type_flag = True, train=train):
    '''
    examples_n: shot number
    samples_n: sample number
    cot_type: choose demonstration type
    '''
    
    assert cot_type in ["simi", "random"]

    if cot_type == "simi":
        # get similar molecules
        simi = top_n_scaffold_similar_molecules(target,list(train['scaffold_fp']), list(train['reactants_smiles']), n=examples_n)
        examples = []
        for i in range(examples_n):
            examples.append((simi[0][i], train.iloc[simi[1][i]]['reaction_type'], train.iloc[simi[1][i]]['products_smiles']))

    elif cot_type == "random":
        examples = []
        random_idx = np.random.randint(0, len(train), examples_n)
        for i in random_idx:
            examples.append((train.iloc[i]['reactants_smiles'], train.iloc[i]['reaction_type'], train.iloc[i]['products_smiles']))

    elif cot_type == "mixed": # TODO
        # 各个种类的反应都有一些
        examples = []

    elif cot_type == "different": # TODO
        # get different molecules
        pass
    
    if reaction_type_flag:
        final_prompt = get_ICL_prompts_reaction_type(target, examples)
    else:
        final_prompt = get_ICL_prompts(target, examples)

    if model == "gpt35":
        response = generate_response_by_gpt35(final_prompt, n=samples_n, temperature=temperature)
    elif model == "gpt4":
        response = generate_response_by_gpt4(final_prompt, n=samples_n, temperature=temperature)
    elif model == "davinci":
        response = generate_response_by_davinci(final_prompt, n=samples_n, temperature=temperature)

    # print('------------------------------------------')
    # print(final_prompt)
    print('------------------------------------------')
    print(response)
    
    if reaction_type_flag:
        try:
            predicted_reaction_type = [r.split('\n')[-2].split(' ')[-1] for r in response]
            products_predict_list = [r.split('\n')[-1].split(' ')[-1] for r in response]
        except Exception as e:
            return response, [], [], 0, []
        
    else:
        products_predict_list = [r.split('\n')[-1].split(' ')[-1] for r in response]
        predicted_reaction_type = ['none']

    # print('------------------------------------------')
    # print(products_predict_list)

    final_predict = max(set(products_predict_list), key = products_predict_list.count)
    num_different_answer = len(set(products_predict_list))

    # print('------------------------------------------')
    # print(final_predict)
    # print('-'*20)
    # print(num_different_answer)

    return response, products_predict_list, final_predict, num_different_answer, predicted_reaction_type

In [141]:
response, products_predict_list, final_predict, num_different_answer, predicted_reaction_type = predict_with_reaction_type("CS(=O)(=O)OC[C@H]1CCC(=O)O1.Fc1ccc(Nc2ncnc3cc(OCCN4CCNCC4)c(OC4CCCC4)cc23)cc1Cl",
                                                                                                  cot_type='random',
                                                                                                  temperature=1,
                                                                                                  reaction_type_flag=False,
                                                                                                  train=train_5reaction)

------------------------------------------
['NC[C@H]1CCC(=O)O1.Fc1ccc(Nc2ncnc3cc(-c4cc(OCCN5CCNCC5)ccc4OC4CCCC4)cc(OCCN4CCNCC4)c23)cc1Cl', 'CS(=O)(=O)OC[C@H]1CCC(=O)O1.Fc1ccc(Nc2ncnc3cc(-c4ccccc4)cc(OCCN4CCNCC4)cc23)cc1Cl', 'CS(=O)(=O)OC[C@H]1CCC(=O)O1.Fc1ccc(Nc2ncnc3cc(-c4ccccc4)cc(OC4CCCC4)cc23)cc1Cl', 'CS(=O)(=O)OC[C@H]1CCC(=O)O1Fc1ccc(-c2cc(Nc3ncnc4cc(-c5ccccn5)cc(OC6CCCC6)cc34)ccc2Cl)cc1', 'CS(=O)(=O)OC[C@H]1CCC(=O)O1.Fc1ccc(Nc2ncnc3cc(-c4cccc5[nH]cnn45)cc2OC2CCCC2)cc1Cl']


In [71]:
products_predict_list

['CS(=O)(=O)OC[C@@H]1CCC(=O)O1.Oc1ccc(Nc2ncnc3cc(OCCN4CCNCC4)c(OC4CCCC4)cc23)cc1Cl',
 'CS(=O)(=O)OC[C@H]1CCC(=O)O1.Fc1ccc(Nc2ncnc3cc(OCCN4CCNCC4)c(OC4CCCC4)cc23)cc1Cl',
 'CS(=O)(=O)OC[C@@H]1CCC(=O)O1.OCCN(CC)CC(=O)c1ccc(Nc2ncnc3cc(OCCN4CCNCC4)c(OC4CCCC4)cc23)cc1Cl',
 'Fc1ccc(Nc2ncnc3cc(OCCN4CCNCC4)c(OC4CCCC4)cc23)cc1Cl.O=C1CCC(=O)O[C@@H]1OC(=O)OC(C)(C)S(=O)(=O)C',
 'Fc1ccc(Nc2ncnc3cc(OCCN4CCNCC4)c(OC4CCCC4)cc23)cc1Cl.CS(=O)(=O)OC[C@@H]1CCCC(=O)O1']

In [72]:
for r in predicted_reaction_type:
    print (r)

none


##### evaluation

In [142]:
def experiment_eval_reation_type(test_number = 10, model = "gpt35", cot_type = "simi", examples_n=20, samples_n=5, tempreture=1, reaction_type_flag = True,
                                 test = test, train=train):
    # query
    correct = 0
    valid_count = 0
    correct_top_1 = 0
    correct_reaction_type_rate_list = []

    test_ = test.sample(test_number, random_state=42) # TODO
    for idx, row in tqdm(test_.iterrows()):
        reactant = row['reactants_smiles']
        product = row['products_smiles']
        reaction_type = row['reaction_type']
        # index = row['index']
        
        print('begin!',idx)
        response, products_predict_list, final_predict, num_different_answer, predicted_reaction_type = predict_with_reaction_type(reactant, model = model, cot_type = cot_type, 
                                                                                                          examples_n=examples_n, samples_n=samples_n,
                                                                                                          temperature=tempreture,
                                                                                                          reaction_type_flag = reaction_type_flag,
                                                                                                          train=train)
        if products_predict_list == []: # invalid answer
            continue

        correct_flag = False
        
        for pred in products_predict_list:
            try:
                mol = Chem.MolFromSmiles(pred)
                pred = Chem.MolToSmiles(mol)
            except Exception as e:
                continue
            
            valid_count += 1
            pred_list = pred.split(".")
            if product in pred_list:
                correct_flag = True
                break
        
        if final_predict == product:
            correct_top_1 += 1
        if correct_flag == True:
            correct += 1

        correct_reaction_type = 0 
        for r in predicted_reaction_type:
            if r == reaction_type:
                correct_reaction_type += 1
        correct_reaction_type_rate = correct_reaction_type/len(predicted_reaction_type)
        correct_reaction_type_rate_list.append(correct_reaction_type_rate)

    top_1_acc = correct_top_1/len(test_)
    top_n_acc = correct/len(test_)
    valid_rate = valid_count/(len(test_)*samples_n)
    mean_correct_reaction_type_rate = np.mean(correct_reaction_type_rate_list)
    return top_1_acc, top_n_acc, valid_rate, mean_correct_reaction_type_rate


In [75]:
top_1_acc, top_n_acc, valid_rate, mean_correct_reaction_type_rate = experiment_eval_reation_type(test_number = 5, model = "davinci", examples_n=2, samples_n=1, reaction_type_flag = False)

0it [00:00, ?it/s]

begin! 46998


1it [00:04,  4.17s/it]

------------------------------------------
['CCOC(=O)c1cn(C2CC2)c2c(C#CC[C@@H]3CCCN3C(=O)OC(C)(C)C)c(F)c(F)cc2c1=O']
begin! 45329


2it [00:07,  3.68s/it]

...

------------------------------------------
['CC#CCOc1ccc(C(=O)NC[C@@H](C(=O)OC)N2CCCC(C)C2)cc1']
begin! 45041


4it [00:14,  3.52s/it]

------------------------------------------
['COC(C(C)(C)CCCCC#C[Si](C)(C)C)c1ccccc1C']
begin! 49475


5it [00:19,  3.82s/it]

------------------------------------------
['Cc1cc(C(=O)N2CCN(C(=O)OC(C)(C)C)C[C@H]2C=O)c(-c2ccccc2)n1-c1ccccc1']


In [76]:
top_1_acc, top_n_acc, valid_rate, mean_correct_reaction_type_rate

(0.2, 0.2, 1.0, 0.0)

In [59]:
top_1_acc, top_n_acc, valid_rate, mean_correct_reaction_type_rate

(0.0, 0.2, 0.66, 0.1)

In [80]:
# log = pd.DataFrame(columns=['dataset_name', 'model', 'cot_type', 'examples_n', 'samples_n', 
#                              'temperature',
#                              'top_1_acc', 'top_n_acc', 'valid_rate'])
# log.loc[len(log)] = [1, 2, 3, 4, 5, 5, 5, 5, 5]
# log
# log.to_csv(f'log/1.csv', index=False)

In [78]:
# 用USPTO作为训练集，USPTO作为测试集
log = pd.DataFrame(columns=['dataset_name', 'model', 'cot_type', 'test_number','examples_n', 'samples_n', 
                             'temperature',
                             'top_1_acc', 'top_n_acc', 'valid_rate', 'mean_correct_reaction_type_rate'])

model = ["davinci"]
cot_type = ['simi']
examples_n = [20]
tempreture = [0.5]
samples_n = [5]

for m in model:
    for c in cot_type:  
                for t in tempreture:
                    top_1_acc, top_n_acc, valid_rate, mean_correct_reaction_type_rate = experiment_eval_reation_type(test_number = 20, model = m, cot_type = c, 
                                                                                                                     examples_n = 20, samples_n=5, tempreture=t,
                                                                                                                     reaction_type_flag = False)
                    log.loc[len(log)] = [dataset_name, m, c, 20, 20, 5, t, top_1_acc, top_n_acc, valid_rate, mean_correct_reaction_type_rate]
                    log.to_csv(f'log/{dataset_name}_{m}_{c}_{t}.csv', index=False)

log.to_csv(f'log/no_reactiontype.csv', index=False)

0it [00:00, ?it/s]

begin! 46998


[08:33:55] Can't kekulize mol.  Unkekulized atoms: 5 6 11 12 24 26 28 30 31
[08:33:55] Can't kekulize mol.  Unkekulized atoms: 5 6 11 12 28 30 32 36 37
[08:33:55] Can't kekulize mol.  Unkekulized atoms: 5 6 11 12 15 36 38 40 41
[08:33:55] Explicit valence for atom # 14 N, 4, is greater than permitted
1it [00:05,  5.65s/it]

------------------------------------------
['CCOC(=O)c1cn(C2CC2)c2c(C#CC[C@@H]3CCCN3C(=O)OC(C)(C)C)c(F)cc(F)c2c1=O', 'CCOC(=O)c1cn(C2CC2)c2c(C#CC[C@@H]3CCCN3C(=O)O)c(F)c(N)c(F)cc2c1=O', 'CCOC(=O)c1cn(C2CC2)c2c(C#CC[C@@H]3CCCN3C(=O)OC(C)(C)C)c(F)c(N)c([N+](=O)[O-])cc2c1=O', 'CCOC(=O)c1cn(C2CC2)c2c(C#CC[C@@H]3CCCN3C(=O)OC(C)(C)C)c(F)c(N)cc2c1=O', 'CCOC(=O)c1cn(C2CC2)c2c(C#CC[C@@H]3CCCN3C(=O)OC(C)(C)C)c(F)c(F)cc2c1=O', 'CCOC(=O)c1cn(C2CC2)c2c(OC)c(N3CCCN(C#CC4CCCN4C(=O)OC(C)(C)C)C3)c(F)c(F)cc2c1=O', 'CCOC(=O)c1cn(C2CC2)c2c(C#CC3CCN(C(=O)OC(C)(C)C)CN3)c(F)c(F)cc2c1=O', 'CCOC(=O)c1cn(C2CC2)c2c(C#CC[C@@H]3CCCN3C(=O)O)c(F)c(F)cc2c1=O', 'CCOC(=O)c1cn(C2CC2)c2c(C#CC[C@@H]3CCCN3C(=O)OC(C)(C)C)c(N)c(F)cc2c1=O', 'CCOC(=O)c1cn(C2CC2)c2c(C#N[C@@H]3CCCN3C(=O)OC(C)(C)C)c(F)c(F)cc2c1=O']
begin! 45329


[08:33:59] Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 15 16 19 20 21 23 25
[08:33:59] Can't kekulize mol.  Unkekulized atoms: 10 11 12 14 16
2it [00:09,  4.75s/it]

...

------------------------------------------
['CC(C)(C)OC(=O)NC1CCC(c2ccc(Br)cc2)CC1', 'CC(C)(C)OC(=O)NC1CCC(c2ccc(Br)cc2)CC1', 'CC(C)(C)OC(=O)N1CCC(O)(c2ccc(Br)cc2)CC1', 'CC(C)(C)OC(=O)N1CCC(O)(c2ccc(Br)cc2)CC1', 'CC(C)(C)OC(=O)N1CCC(c2ccc(Br)cc2)CC1', 'CC(C)(C)OC(=O)N1CCC(c2ccc(Br)cc2)CC1', 'CC(C)(C)OC(=O)N1CCC(c2ccc(Br)cc2)CC1', 'CC(C)(C)OC(=O)N1CCC(O)(c2ccc(Br)cc2)CC1', 'CC(C)(C)OC(=O)NC1(c2ccc(Br)cc2)CC1', 'CC(C)(C)OC(=O)N1CCC(O)(c2ccc(Br)cc2)CC1']
begin! 47898


19it [01:20,  3.89s/it]

------------------------------------------
['CC#CCOc1cc(Oc2c(F)cccc2F)ncn1', 'CC#CCOc1cc(Cl)nc(Oc2c(F)cccc2F)n1', 'CC#CCOc1cc(Cl)nc(Oc2c(F)cccc2F)n1', 'Oc1c(F)cccc1-c1cc(CC#CCO)ncn1Cl', 'CC#CCOc1cc(Cl)nc[n+]1[O-]', 'Oc1cc(Cl)cc(-c2ccc(F)cn2)c1CC#CCO', 'Oc1c(F)cccc1CC#CCOc1cc(Cl)ncn1', 'CC#C[C@@H]Oc1cc(Cl)ncn1Oc1c(F)cccc1F', 'CC#CCOc1cc(Cl)nc(Oc2c(F)cccc2F)n1', 'CC#CCOc1cc(Oc2ccccc2F)ncn1']
begin! 47981


[08:35:15] Can't kekulize mol.  Unkekulized atoms: 19 20 21 22 24
[08:35:15] Can't kekulize mol.  Unkekulized atoms: 19 20 21 22 24
[08:35:15] Can't kekulize mol.  Unkekulized atoms: 8 9 10 11 12 16 17 33 34
[08:35:15] Can't kekulize mol.  Unkekulized atoms: 19 20 21 22 23
20it [01:26,  4.32s/it]

------------------------------------------
['CC(C)C(=O)N1CCC(c2c[nH]c3c(C(N)=O)cc(-c4ccc(Br)cs4)cc23)CC1', 'CC(C)C(=O)N1CCC(c2c[nH]c3c(C(N)=O)cc(-c4cccs4)cc23)CC1', 'CC(C)C(=O)N1CCC(c2c[nH]c3c(C(N)=O)cc(-c4cccs4)cc23)CC1', 'CC(C)C(=O)c1cc(Br)cs1.CCS(=O)(=O)N1CCC(c2c[nH]c3c(C(N)=O)cc(-c4ccc5c(c4)CNC5)cc23)CC1', 'CCS(=O)(=O)N1CCC(c2c[nH]c3c(C(N)=O)cc(-c4cccs4)cc23)CC1', 'CC(C)C(=O)N1CCC(c2c[nH]c3c(C(N)=O)cc(-c4ccc(Br)cs4)cc23)CC1', 'CCS(=O)(=O)N1CCC(c2c[nH]c3c(C(N)=O)cc(-c4cccc(C(O)=CBr)c4)cc23)CC1', 'CCS(=O)(=O)N1CCC(c2c[nH]c3c(C(N)=O)cc(-c4cc(C(N)=O)s5ccc(Br)c45)cc23)CC1', 'CC(C)C(=O)c1cc(-c2ccc3c(C(N)=O)cc(C4CCN(S(=O)(=O)CCCN(C)C)CC4)c3c2)cs1', 'CCS(=O)(=O)N1CCC(c2c[nH]c3c(C(N)=O)cc(-c4ccccs4)cc23)CC1']


In [144]:
# 用USPTO作为训练集，5reaction作为测试集
log = pd.DataFrame(columns=['dataset_name', 'model', 'cot_type', 'test_number', 'examples_n', 'samples_n', 
                             'temperature',
                             'top_1_acc', 'top_n_acc', 'valid_rate', 'mean_correct_reaction_type_rate'])

model = ["davinci"]
cot_type = ['simi']
examples_n = [20]
tempreture = [0.5]
samples_n = [5]

for m in model:
    for c in cot_type:  
                for t in tempreture:
                    top_1_acc, top_n_acc, valid_rate, mean_correct_reaction_type_rate = experiment_eval_reation_type(test_number = 20, model = m, cot_type = c, 
                                                                                                                     examples_n = 20, samples_n=5, tempreture=t,
                                                                                                                     reaction_type_flag = False,
                                                                                                                     test=test_5reaction, train=train)
                    log.loc[len(log)] = [dataset_name, m, c, 20, 20, 5, t, top_1_acc, top_n_acc, valid_rate, mean_correct_reaction_type_rate]
                    log.to_csv(f'log/uspto_for_5reaction/{dataset_name}_{m}_{c}_{t}.csv', index=False)

log.to_csv(f'log/no_reactiontype.csv', index=False)

0it [00:00, ?it/s]

begin! 7025


[04:26:52] Can't kekulize mol.  Unkekulized atoms: 35 36 37 38 39 40 50
1it [00:09,  9.18s/it]

------------------------------------------
['COC(=O)[C@@H]1C2CCC(CC2)[C@H]1Nc1cc(-c2ccc(C(C)(C)C)cc2)nc(-c2ccc(S(=O)(=O)n3cc(C)c4cncnc43)cc2)n1', 'COC(=O)[C@@H]1C2CCC(CC2)[C@H]1Nc1cc(-c2ccc(C(C)(C)C)cc2)nc(-c2cnc3c(c2)c(-c2cccc4cc[nH]c24)cn3S(=O)(=O)c2ccc(C)cc2)n1', 'COC(=O)[C@@H]1C2CCC(CC2)[C@H]1Nc1cc(-c2ccc(C(C)(C)C)cc2)nc(-c2cc(S(=O)(=O)n3ccc4c(c3)c(B3OC(C)(C)C(C)(C)O3)cn4)cnc2)n1', 'COC(=O)[C@@H]1C2CCC(CC2)[C@H]1Nc1cc(-c2ccc(-c3cnc(N)cn3)cc2)nc(Cl)n1', 'COC(=O)[C@@H]1C2CCC(CC2)[C@H]1Nc1cc(-c2ccc(C(C)(C)C)cc2)nc(-c2cc(S(=O)(=O)n3ccc4c(c3)c(B3OC(C)(C)C(C)(C)O3)cn4Cl)cnc2)n1']
begin! 8827


2it [00:13,  6.14s/it]

...

------------------------------------------
['OB(O)c1cccc(C(=C(c2ccccc2)c2ccccc2)c2ccccc2)c1F', 'OB(O)c1cccc(C(=C(c2ccccc2)c2ccccc2)c2ccccc2)c1F', 'OB(O)c1cccc(C(=C(c2ccccc2)c2ccccc2)c2ccccc2)c1F', 'OB(O)c1cccc(/C=C/Br)c1F', 'OB(O)c1cccc(C(=C(c2ccccc2)c2ccccc2)c2ccccc2)c1F']
begin! 8727


19it [01:35,  4.26s/it]

------------------------------------------
['O=C(Nc1cccc(-c2nnc3n2CCCC3)n1)c1cc(-n2cnc(C3CC3)c2-c2cccnc2)c(Cl)cc1F', 'O=C(Nc1cccc(-c2nnc3n2CCCC3)n1)c1cc(-n2cnc(C3CC3)c2-c2cccnc2)c(Cl)cc1F', 'O=C(Nc1cccc(-c2nnc3n2CCCC3)n1)c1cc(-n2cnc(C3CC3)c2-c2cccnc2)c(Cl)cc1F', 'O=C(Nc1cccc(-c2nnc3n2CCCC3)n1)c1cc(-n2cnc(C3CC3)c2-c2cccnc2)c(Cl)cc1F', 'O=C(Nc1cccc(-c2nnc3n2CCCC3)n1)c1cc(-n2cnc(C3CC3)c2-c2cccnc2)c(Cl)cc1F']
begin! 6778


20it [01:39,  4.97s/it]

------------------------------------------
['CC(Oc1cc(-c2ccc(OCc3ccccc3)c(C(=O)Nc3cccnc3)c2)cnc1N)c1ccc(Cl)cc1', 'CC(Oc1cc(-c2ccc(Cl)cc2)cnc1N)c1ccc(Br)cc1', 'CC(Oc1cc(-c2cncc(N[C@@H](CO)c3ccccc3)c2)cnc1N)c1ccc(Cl)cc1', 'CC(Oc1cc(-c2ccc(Nc3ccncn3)cc2)cnc1)c1ccc(Cl)cc1', 'CC(Oc1cc(-c2ccc(OCc3ccccc3)c(C(=O)Nc3cccnc3)c2)cnc1N)c1ccc(Cl)cc1']


#### ONLY CREATE PROMPTS

In [119]:
def create_prompts_with_reactiontype(target, cot_type = "simi", examples_n=20, reactiontype_flag= 1, train = train):
    '''
    examples_n: shot number
    samples_n: sample number
    cot_type: choose demonstration type
    '''
    
    assert cot_type in ["simi", "random"]

    if cot_type == "simi":
        # get similar molecules
        simi = top_n_scaffold_similar_molecules(target,list(train['scaffold_fp']), list(train['reactants_smiles']), n=examples_n)
        examples = []
        for i in range(examples_n):
            examples.append((simi[0][i], train.iloc[simi[1][i]]['reaction_type'], train.iloc[simi[1][i]]['products_smiles']))

    elif cot_type == "random":
        examples = []
        random_idx = np.random.randint(0, len(train), examples_n)
        for i in random_idx:
            examples.append((train.iloc[i]['reactants_smiles'], train.iloc[i]['reaction_type'], train.iloc[i]['products_smiles']))

    elif cot_type == "mixed": # TODO
        # 各个种类的反应都有一些
        examples = []

    elif cot_type == "different": # TODO
        # get different molecules
        pass
    
    if reactiontype_flag==1:
        final_prompt = get_ICL_prompts_reaction_type(target, examples)
    elif reactiontype_flag==0:
        final_prompt = get_ICL_prompts(target, examples)
    else:
        final_prompt = get_ICL_prompts_reaction_type_desciption(target, examples)

    return final_prompt

In [36]:
# USPTO_50K
for reaction_flag in [0,1,2]:
    for s in [5,10]:
        for type in ['random','simi']:
            valid_prompt = pd.DataFrame(columns=['reactant', 'product', 'reactiontype', 'prompt'])
            for idx, row in tqdm(valid.iterrows()):
                reactant = row['reactants_smiles']
                product = row['products_smiles']
                ra = row['reaction_type']
                prompt = create_prompts_with_reactiontype(reactant, cot_type=type, examples_n=s, reactiontype_flag=reaction_flag)
                valid_prompt.loc[len(valid_prompt)] = [reactant, product, ra, prompt]

            valid_prompt.to_csv(f'prompts/valid_prompt_{type}_sample{s}_reaction_{reaction_flag}.csv', index=False)
        

30it [00:00, 819.75it/s]
30it [00:02, 12.62it/s]
30it [00:00, 719.46it/s]
30it [00:02, 13.00it/s]
30it [00:00, 1074.88it/s]
30it [00:02, 13.36it/s]
30it [00:00, 723.04it/s]
30it [00:02, 12.96it/s]
30it [00:00, 880.27it/s]
30it [00:02, 13.20it/s]
30it [00:00, 718.14it/s]
30it [00:02, 12.81it/s]


In [135]:
train_5reaction['reaction_type'].value_counts()

reaction_type
Suzuki              8231
Kumada               866
Chan-Lam             475
Buchwald-Hartwig     392
Hiyama               188
Name: count, dtype: int64

In [138]:
# 5 reaction

for reaction_flag in [0,1]:
    for s in [10]:
        for type in ['simi']:
            for reaction_t in ['Buchwald-Hartwig', 'Chan-Lam', 'Hiyama', 'Kumada', 'Suzuki']:

                valid_prompt = pd.DataFrame(columns=['reactant', 'product', 'reactiontype', 'prompt'])

                c = 0
                for idx, row in tqdm(test_5reaction.iterrows()):
                    ra = row['reaction_type']
                    if ra!=reaction_t:
                        continue

                    reactant = row['reactants_smiles']
                    product = row['products_smiles']
                    
                    prompt = create_prompts_with_reactiontype(reactant, cot_type=type, examples_n=s, 
                                                            reactiontype_flag=reaction_flag, train=train)
                    valid_prompt.loc[len(valid_prompt)] = [reactant, product, ra, prompt]

                    c+=1
                    if c>=25:
                        break

                valid_prompt.to_csv(f'prompts/5reaction_uspto_trained/valid_prompt_{type}_sample{s}_reaction_{reaction_flag}_type_{reaction_t}.csv', index=False)
            

24it [00:02, 10.56it/s]
70it [00:01, 35.80it/s] 
1130it [00:01, 834.36it/s]
135it [00:01, 72.38it/s] 
227it [00:02, 79.38it/s]  
24it [00:03,  7.85it/s]
70it [00:03, 21.39it/s] 
1130it [00:02, 490.83it/s]
135it [00:03, 35.35it/s] 
227it [00:04, 55.36it/s]  


### 2.多次query with different demonstration

# 三、区分催化剂

dataset construct

In [221]:
def get_reagent_reactants(reactants_smiles, products_smiles):
    # atom set
    products_atom_set = get_atom_set(products_smiles)

    # 解析反应物和产物的SMILES字符串
    reactants = [Chem.MolFromSmiles(smi) for smi in reactants_smiles.split('.')]
    products = [Chem.MolFromSmiles(smi) for smi in products_smiles.split('.')]

    reagents = []
    # 对每个反应物进行判断
    for reactant in reactants:
        reactants_atom_set = get_atom_set(Chem.MolToSmiles(reactant))
        is_reagent = False

        # 检查反应物是否出现在产物中且未发生改变
        for product in products:
            if product.HasSubstructMatch(reactant):
                is_reagent = True
                break
        
        # 如果反应物分子中的每一个原子都没有在product中出现，那么这个反应物分子就是催化剂
        if len(reactants_atom_set.intersection(products_atom_set))==0:
            is_reagent = True
            
        if is_reagent:
            reagents.append(Chem.MolToSmiles(reactant))


    return '.'.join(reagents), '.'.join([Chem.MolToSmiles(reactant) for reactant in reactants if Chem.MolToSmiles(reactant) not in reagents])



In [222]:
get_reagent_reactants(separate_reagents_5reaction['reactants_smiles'].iloc[0], separate_reagents_5reaction['products_smiles'].iloc[0])

('CC(C)(C(=O)Nc1nncs1)C(c1ccccc1)c1ccc2[nH]ncc2c1.[K+].[K+].[K+]',
 'Ic1ccccc1.O=P([O-])([O-])[O-]')

In [202]:
import pandas as pd
from rdkit import Chem

def get_atom_set(smiles):
    mol = Chem.MolFromSmiles(smiles)
    atom_set = set(atom.GetSymbol() for atom in mol.GetAtoms())
    return atom_set

def classify_reagents(data):
    # 创建新的列
    data['Reagents_smiles'] = ''
    data['reactants_smiles_only'] = ''

    # 遍历每一行数据
    for index, row in data.iterrows():
        reactants_smiles = row['reactants_smiles']
        products_smiles = row['products_smiles']

        # atom set
        products_atom_set = get_atom_set(products_smiles)

        # 解析反应物和产物的SMILES字符串
        reactants = [Chem.MolFromSmiles(smi) for smi in reactants_smiles.split('.')]
        products = [Chem.MolFromSmiles(smi) for smi in products_smiles.split('.')]

        reagents = []
        # 对每个反应物进行判断
        for reactant in reactants:
            reactants_atom_set = get_atom_set(Chem.MolToSmiles(reactant))
            is_reagent = False

            # 检查反应物是否出现在产物中且未发生改变
            for product in products:
                if product.HasSubstructMatch(reactant):
                    is_reagent = True
                    break
            
            # 如果反应物分子中的每一个原子都没有在product中出现，那么这个反应物分子就是催化剂
            if len(reactants_atom_set.intersection(products_atom_set))==0:
                is_reagent = True
                
            if is_reagent:
                reagents.append(Chem.MolToSmiles(reactant))

        # 更新Reagents_smiles列
        data.at[index, 'Reagents_smiles'] = '.'.join(reagents)
        data.at[index, 'reactants_smiles_only'] = '.'.join([Chem.MolToSmiles(reactant) for reactant in reactants if Chem.MolToSmiles(reactant) not in reagents])

    # 返回结果
    return data


In [253]:
separate_reagents_5reaction = classify_reagents(df_5reaction)

[07:51:40] WARNING: not removing hydrogen atom without neighbors
[07:51:40] WARNING: not removing hydrogen atom without neighbors
[07:51:43] WARNING: not removing hydrogen atom without neighbors
[07:51:43] WARNING: not removing hydrogen atom without neighbors
[07:51:43] WARNING: not removing hydrogen atom without neighbors
[07:51:43] WARNING: not removing hydrogen atom without neighbors
[07:51:44] WARNING: not removing hydrogen atom without neighbors
[07:51:44] WARNING: not removing hydrogen atom without neighbors
[07:51:44] WARNING: not removing hydrogen atom without neighbors
[07:51:44] WARNING: not removing hydrogen atom without neighbors


In [267]:
separate_reagents_uspto = classify_reagents(df)

In [254]:
separate_reagents_5reaction[separate_reagents_5reaction['Reagents_smiles']!='']

,reactants_smiles,products_smiles,reaction_type,scaffold_fp,set,Reagents_smiles,reactants_smiles_only
0,CC(C)(C(=O)Nc1nncs1)C(c1ccccc1)c1ccc2[nH]ncc2c...,CC(C)(C(=O)Nc1nncs1)C(c1ccccc1)c1ccc2c(cnn2-c2...,Buchwald-Hartwig,<rdkit.DataStructs.cDataStructs.UIntSparseIntV...,test,CC(C)(C(=O)Nc1nncs1)C(c1ccccc1)c1ccc2[nH]ncc2c...,Ic1ccccc1.O=P([O-])([O-])[O-]
1,COC(=O)C1(C)CCNCC1.Cc1cncc(Br)n1.Cl.O=C(/C=C/c...,COC(=O)C1(C)CCN(c2cncc(C)n2)CC1,Buchwald-Hartwig,<rdkit.DataStructs.cDataStructs.UIntSparseIntV...,train,COC(=O)C1(C)CCNCC1.Cl.[Pd].[Pd].[Cs+].[Cs+],Cc1cncc(Br)n1.O=C(/C=C/c1ccccc1)/C=C/c1ccccc1....
2,CNc1ccccc1.COc1cc(Br)cc(OC)c1.O=C(/C=C/c1ccccc...,COc1cc(OC)cc(N(C)c2ccccc2)c1,Buchwald-Hartwig,<rdkit.DataStructs.cDataStructs.UIntSparseIntV...,train,CNc1ccccc1.[Pd].[Pd].[Na+],COc1cc(Br)cc(OC)c1.O=C(/C=C/c1ccccc1)/C=C/c1cc...
3,Brc1ccc(-c2ccccc2)cc1.c1ccc(-c2nc3ccc4c([nH]c5...,c1ccc(-c2ccc(-n3c4c(ccc5nc(-c6ccccc6)sc54)c4c5...,Buchwald-Hartwig,<rdkit.DataStructs.cDataStructs.UIntSparseIntV...,train,c1ccc(-c2nc3ccc4c([nH]c5c6ccccc6c6ccccc6c45)c3...,Brc1ccc(-c2ccccc2)cc1.O=C(/C=C/c1ccccc1)/C=C/c...
4,CNc1ccccc1.O=c1cc(-c2ccccc2)oc2ccc(Br)cc12.O=C...,CN(c1ccccc1)c1ccc2oc(-c3ccccc3)cc(=O)c2c1,Buchwald-Hartwig,<rdkit.DataStructs.cDataStructs.UIntSparseIntV...,train,CNc1ccccc1.[Pd].[Pd].[Na+],O=c1cc(-c2ccccc2)oc2ccc(Br)cc12.O=C(/C=C/c1ccc...
...,...,...,...,...,...,...,...
11337,CC(C)(C)OC(=O)Nc1cc(B2OC(C)(C)C(C)(C)O2)ccn1.C...,CN(C)CC1CCN(c2ccccc2NC(=O)c2nc(-c3ccnc(NC(=O)O...,Suzuki,<rdkit.DataStructs.cDataStructs.UIntSparseIntV...,test,[Na+].[Na+],CC(C)(C)OC(=O)Nc1cc(B2OC(C)(C)C(C)(C)O2)ccn1.C...
11347,CC(C)(C)OC(=O)N1CCC(n2cc(Br)c(-c3ccncc3)n2)CC1...,CC(C)(C)OC(=O)N1CCC(n2cc(-c3ccc4c(c3)CCC4=O)c(...,Suzuki,<rdkit.DataStructs.cDataStructs.UIntSparseIntV...,test,[Pd].[K+].[K+],CC(C)(C)OC(=O)N1CCC(n2cc(Br)c(-c3ccncc3)n2)CC1...
11356,CC1=NC(c2ccc(F)c(Br)c2)(C2CC2)N=C1N.OB(O)c1cnc...,CC1=NC(c2ccc(F)c(-c3cncc(Cl)c3)c2)(C2CC2)N=C1N,Suzuki,<rdkit.DataStructs.cDataStructs.UIntSparseIntV...,train,[K+].[K+],CC1=NC(c2ccc(F)c(Br)c2)(C2CC2)N=C1N.OB(O)c1cnc...
11357,FC(F)(F)c1cc(N2CCOCC2)nc2c(Br)cnn12.OB(O)c1ccc...,FC(F)(F)c1cc(N2CCOCC2)nc2c(-c3ccccc3)cnn12,Suzuki,<rdkit.DataStructs.cDataStructs.UIntSparseIntV...,train,[K+].[K+],FC(F)(F)c1cc(N2CCOCC2)nc2c(Br)cnn12.OB(O)c1ccc...


In [280]:
separate_reagents_uspto[separate_reagents_uspto['Reagents_smiles']!=''][['reactants_smiles','products_smiles','Reagents_smiles']].head(2)

,reactants_smiles,products_smiles,Reagents_smiles
0,CS(=O)(=O)OC[C@H]1CCC(=O)O1.Fc1ccc(Nc2ncnc3cc(...,O=C1CC[C@H](CN2CCN(CCOc3cc4ncnc(Nc5ccc(F)c(Cl)...,Fc1ccc(Nc2ncnc3cc(OCCN4CCNCC4)c(OC4CCCC4)cc23)...
4,Fc1cc2c(Cl)ncnc2cn1.NC1CCCCCC1,Fc1cc2c(NC3CCCCCC3)ncnc2cn1,NC1CCCCCC1


load separate dataset

In [288]:
separate_data = pd.read_pickle("data/reaction_prediction/uspto/uspto_sep.pickle")

In [289]:
separate_data

,reactants_mol,products_mol,reagents_mol,set
0,<rdkit.Chem.rdchem.Mol object at 0x000001F24AF...,<rdkit.Chem.rdchem.Mol object at 0x000001F3DF4...,<rdkit.Chem.rdchem.Mol object at 0x000001F532F...,train
1,<rdkit.Chem.rdchem.Mol object at 0x000001F24AF...,<rdkit.Chem.rdchem.Mol object at 0x000001F3DF4...,<rdkit.Chem.rdchem.Mol object at 0x000001F532F...,train
2,<rdkit.Chem.rdchem.Mol object at 0x000001F24AF...,<rdkit.Chem.rdchem.Mol object at 0x000001F3DF4...,<rdkit.Chem.rdchem.Mol object at 0x000001F532F...,train
3,<rdkit.Chem.rdchem.Mol object at 0x000001F24AF...,<rdkit.Chem.rdchem.Mol object at 0x000001F3DF4...,<rdkit.Chem.rdchem.Mol object at 0x000001F532F...,train
4,<rdkit.Chem.rdchem.Mol object at 0x000001F24AF...,<rdkit.Chem.rdchem.Mol object at 0x000001F3DF4...,<rdkit.Chem.rdchem.Mol object at 0x000001F532F...,train
...,...,...,...,...
479030,<rdkit.Chem.rdchem.Mol object at 0x000001F3DF4...,<rdkit.Chem.rdchem.Mol object at 0x000001F532F...,<rdkit.Chem.rdchem.Mol object at 0x000001F47D0...,test
479031,<rdkit.Chem.rdchem.Mol object at 0x000001F3DF4...,<rdkit.Chem.rdchem.Mol object at 0x000001F532F...,<rdkit.Chem.rdchem.Mol object at 0x000001F47D0...,test
479032,<rdkit.Chem.rdchem.Mol object at 0x000001F3DF4...,<rdkit.Chem.rdchem.Mol object at 0x000001F532F...,<rdkit.Chem.rdchem.Mol object at 0x000001F47D0...,test
479033,<rdkit.Chem.rdchem.Mol object at 0x000001F3DF4...,<rdkit.Chem.rdchem.Mol object at 0x000001F532F...,<rdkit.Chem.rdchem.Mol object at 0x000001F47D0...,test


In [290]:
separate_data['reactants_smiles'] = separate_data['reactants_mol'].apply(lambda x: Chem.MolToSmiles(x))
separate_data['products_smiles'] = separate_data['products_mol'].apply(lambda x: Chem.MolToSmiles(x))
separate_data['Reagents_smiles'] = separate_data['reagents_mol'].apply(lambda x: Chem.MolToSmiles(x))
# 获取脚手架指纹
separate_data['scaffold_fp'] = separate_data['reactants_smiles'].apply(lambda x: get_scaffold_fp(x))

In [293]:
separate_data_train = separate_data[separate_data['set']=='train'][['reactants_smiles','products_smiles','Reagents_smiles','scaffold_fp']]
separate_data_test = separate_data[separate_data['set']=='test'][['reactants_smiles','products_smiles','Reagents_smiles','scaffold_fp']]

In [295]:
separate_data_test

,reactants_smiles,products_smiles,Reagents_smiles,scaffold_fp
439035,N#Cc1ccsc1N.O=[N+]([O-])c1cc(F)c(F)cc1F,N#Cc1ccsc1Nc1cc(F)c(F)cc1[N+](=O)[O-],C1CCOC1.[H-].[Na+],<rdkit.DataStructs.cDataStructs.UIntSparseIntV...
439036,COC(=O)Cc1cn(C)c2cc(O)ccc12.Cc1nn(-c2ccc(C(F)(...,COC(=O)Cc1cn(C)c2cc(OCC(C)c3cn(-c4ccc(C(F)(F)F...,CCCCP(CCCC)CCCC.Cc1ccccc1,<rdkit.DataStructs.cDataStructs.UIntSparseIntV...
439037,Cl.NC1CCN(CC2Cn3c(=O)ccc4ncc(F)c2c43)CC1O.O=Cc...,O=c1ccc2ncc(F)c3c2n1CC3CN1CCC(NCc2cc3c(cn2)OCS...,CCOCC.Cl.ClC(Cl)Cl,<rdkit.DataStructs.cDataStructs.UIntSparseIntV...
439038,C=C(C)C(=O)Cl.CC(C)=C1C(=O)N(c2ccc(O)cc2)C(=O)...,C=C(C)C(=O)Oc1ccc(N2C(=O)C(=C(C)C)C(=C(C)c3cc(...,CCN(CC)CC.ClCCl,<rdkit.DataStructs.cDataStructs.UIntSparseIntV...
439039,O=Cc1cncc(Cl)c1COC1CCCCO1,OCc1cncc(Cl)c1COC1CCCCO1,OCc1c(Cl)cncc1Cl,<rdkit.DataStructs.cDataStructs.UIntSparseIntV...
...,...,...,...,...
479030,COC(=O)NCC1Cc2c(Cl)cc3c(c2O1)CCC3,CNCC1Cc2c(Cl)cc3c(c2O1)CCC3,[Al+3].[H-].[H-].[H-].[H-].[Li+],<rdkit.DataStructs.cDataStructs.UIntSparseIntV...
479031,COc1cccc(C(=O)Cl)c1.COc1ccccc1OC,COc1cccc(C(=O)c2ccc(OC)c(OC)c2)c1,COc1ccc(C(=CC#N)c2ccc(OC)c(OC)c2)cc1.ClCCl.[Al...,<rdkit.DataStructs.cDataStructs.UIntSparseIntV...
479032,C#Cc1cccc(C2C(C(=O)OC)=C(C)NC(C(OC)OC)=C2C(=O)...,C#Cc1cccc(C2C(C(=O)OC)=C(C)NC(C=O)=C2C(=O)OC)c1,CC(C)=O.Cl,<rdkit.DataStructs.cDataStructs.UIntSparseIntV...
479033,CO.COc1ccc(CC(=O)c2ccc(O)cc2O)cc1,COc1ccc(CC(=O)c2ccc(OC)cc2O)cc1,,<rdkit.DataStructs.cDataStructs.UIntSparseIntV...


separate prompt design

In [229]:
def get_ICL_prompts_separate1(target, target_product, examples):

    reagent, reactants = get_reagent_reactants(target, target_product)
    prompt = "You are an expert chemist. Given only the reactants and reagents (some examples might not have reagents), predict the major organic reaction products as valid SMILES, following these principle.\n \
    Moleculars are splitted by .  \n \
Focus only on the given starting materials and conditions. Do not infer additional reagents or steps. \
Strive for the simplest and most reasonable organic products. Avoid exotic or improbable outcomes. \
Balance atom counts properly and produce chemically valid SMILES. \
Do not include yields, mechanisms, or explanatory text. Only output the major product SMILES. \n \
Catalysts i.e. Reagents either (i) have none of their constituent atoms in the products, or (ii) appear completely unchanged in the product SMILES."


    for example in examples:
        prompt += f"\nReactants: {example[0]}\nReagents: {example[1]}\nProducts: {example[2]}\n"
    prompt += f"\nReactants: {reactants}\nReagents: {reagent}\nProducts:"
    return prompt

In [247]:
def get_ICL_prompts_separate2(target, examples):
    prompt = "You are an expert chemist. You are expected to predict reaction products based on Reactants+Reagents only, with first considering which molecular(s) in Reactants+Reagents are reagents.\n \
    Moleculars are splitted by . \n \
    Reagents either (i) have none of their constituent atoms in the products, or (ii) appear completely unchanged in the product SMILES. \n \
Focus only on the given starting materials and conditions. Do not infer additional reagents or steps. \
Strive for the simplest and most reasonable organic products. Avoid exotic or improbable outcomes. \
Balance atom counts properly and produce chemically valid SMILES. \
Do not include yields, mechanisms, or explanatory text. Only output the major product SMILES. \n \
You should output two things, Possible Reagents as well as the major product SMILES."

    for example in examples:
        prompt += f"\nReactants+Reagents: {example[0]}\nReagents: {example[1]}\nProducts: {example[2]}\n"
    prompt += f"\nReactants+Reagents: {target}\nReagents:"
    return prompt

generate predict

In [299]:
np.random.seed(42)

def predict_with_seperate1(target, target_product, model = "gpt35", cot_type = "simi", examples_n=20, samples_n=5, temperature=0.5, train=separate_reagents_5reaction):
    '''
    parameters:
        examples_n: shot number
        samples_n: sample number
        cot_type: choose demonstration type
    examples:
        Reactants: {example[0]}\nReagents: {example[1]}\nProducts: {example[2]}\n
    predicts:
        Products only
    '''
    
    assert cot_type in ["simi", "random"]

    if cot_type == "simi":
        # get similar molecules
        simi = top_n_scaffold_similar_molecules(target,list(train['scaffold_fp']), list(train['reactants_smiles']), n=examples_n)
        examples = []
        for i in range(examples_n):
            examples.append((train.iloc[simi[1][i]]['reactants_smiles'], train.iloc[simi[1][i]]['Reagents_smiles'], train.iloc[simi[1][i]]['products_smiles']))

    elif cot_type == "random":
        examples = []
        random_idx = np.random.randint(0, len(train), examples_n)
        for i in random_idx:
            examples.append((train.iloc[i]['reactants_smiles'], train.iloc[i]['Reagents_smiles'], train.iloc[i]['products_smiles']))

    elif cot_type == "mixed": # TODO
        # 各个种类的反应都有一些
        examples = []

    elif cot_type == "different": # TODO
        # get different molecules
        pass
    
    final_prompt = get_ICL_prompts_separate1(target, target_product, examples)


    if model == "gpt35":
        response = generate_response_by_gpt35(final_prompt, n=samples_n, temperature=temperature)
    elif model == "gpt4":
        response = generate_response_by_gpt4(final_prompt, n=samples_n, temperature=temperature)
    elif model == "davinci":
        response = generate_response_by_davinci(final_prompt, n=samples_n, temperature=temperature)

    print('------------------------------------------')
    print(final_prompt)
    print('------------------------------------------')
    print(response)
    
    try:
        products_predict_list = [r.split('\n')[-1].split(' ')[-1] for r in response]
    except Exception as e:
        return response, [], [], 0, []

    # print('------------------------------------------')
    # print(products_predict_list)

    final_predict = max(set(products_predict_list), key = products_predict_list.count)
    num_different_answer = len(set(products_predict_list))

    # print('------------------------------------------')
    # print(final_predict)
    # print('-'*20)
    # print(num_different_answer)

    return response, products_predict_list, final_predict, num_different_answer

In [227]:
print(separate_reagents_5reaction.iloc[0]['reactants_smiles'])
print(separate_reagents_5reaction.iloc[0]['products_smiles'])

CC(C)(C(=O)Nc1nncs1)C(c1ccccc1)c1ccc2[nH]ncc2c1.Ic1ccccc1.O=P([O-])([O-])[O-].[K+].[K+].[K+]
CC(C)(C(=O)Nc1nncs1)C(c1ccccc1)c1ccc2c(cnn2-c2ccccc2)c1


In [228]:
response, products_predict_list, final_predict, num_different_answer = predict_with_seperate1(target='CC(C)(C(=O)Nc1nncs1)C(c1ccccc1)c1ccc2[nH]ncc2c1.Ic1ccccc1.O=P([O-])([O-])[O-].[K+].[K+].[K+]',
                                                                                              target_product='CC(C)(C(=O)Nc1nncs1)C(c1ccccc1)c1ccc2c(cnn2-c2ccccc2)c1')

------------------------------------------
You are an expert chemist. Given only the reactants and reagents (some examples might not have reagents), predict the major organic reaction products as valid SMILES, following these principle.
     Moleculars are splitted by .  
 Focus only on the given starting materials and conditions. Do not infer additional reagents or steps. Strive for the simplest and most reasonable organic products. Avoid exotic or improbable outcomes. Balance atom counts properly and produce chemically valid SMILES. Do not include yields, mechanisms, or explanatory text. Only output the major product SMILES. 
 Catalysts i.e. Reagents either (i) have none of their constituent atoms in the products, or (ii) appear completely unchanged in the product SMILES.
Reactants: Ic1n[nH]c2ccc(NCc3ccccc3)cc12.Nc1cccc(B(O)O)c1
Reagents: 
Products: Nc1cccc(-c2n[nH]c3ccc(NCc4ccccc4)cc23)c1

Reactants: CCNC(=O)c1cc(F)c(C)c(-c2ccc3c(I)n[nH]c3c2)c1.Cc1ccc(B(O)O)cc1
Reagents: 
Products: 

In [233]:
products_predict_list, final_predict, num_different_answer

(['Ic1ccccc1-c1nncs1',
  'Ic1ccccc1-c1nnc2ccc(-c3ccccc3)cc12',
  'Ic1ccccc1OP(=O)([O-])([O-])[O-]',
  'Ic1ccccc1-c1nncs1',
  'Ic1ccccc1-c1nncs1'],
 'Ic1ccccc1-c1nncs1',
 3)

In [241]:
np.random.seed(42)

def predict_with_seperate2(target, model = "gpt35", cot_type = "simi", examples_n=20, samples_n=5, temperature=0.5, train=separate_reagents_5reaction):
    '''
    parameters:
        examples_n: shot number
        samples_n: sample number
        cot_type: choose demonstration type
    examples:
        Reactants+Reagents: {example[0]}\nReagents: {example[1]}Products: {example[2]}\n
    predicts:
        Reagents + Products
    '''
    
    assert cot_type in ["simi", "random"]

    if cot_type == "simi":
        # get similar molecules
        simi = top_n_scaffold_similar_molecules(target,list(train['scaffold_fp']), list(train['reactants_smiles']), n=examples_n)
        examples = []
        for i in range(examples_n):
            examples.append((train.iloc[simi[1][i]]['reactants_smiles'], train.iloc[simi[1][i]]['Reagents_smiles'], train.iloc[simi[1][i]]['products_smiles']))

    elif cot_type == "random":
        examples = []
        random_idx = np.random.randint(0, len(train), examples_n)
        for i in random_idx:
            examples.append((train.iloc[i]['reactants_smiles'], train.iloc[i]['Reagents_smiles'], train.iloc[i]['products_smiles']))

    elif cot_type == "mixed": # TODO
        # 各个种类的反应都有一些
        examples = []

    elif cot_type == "different": # TODO
        # get different molecules
        pass
    
    final_prompt = get_ICL_prompts_separate2(target, examples)


    if model == "gpt35":
        response = generate_response_by_gpt35(final_prompt, n=samples_n, temperature=temperature)
    elif model == "gpt4":
        response = generate_response_by_gpt4(final_prompt, n=samples_n, temperature=temperature)
    elif model == "davinci":
        response = generate_response_by_davinci(final_prompt, n=samples_n, temperature=temperature)

    print('------------------------------------------')
    print(final_prompt)
    print('------------------------------------------')
    print(response)
    
    try:
        Reagents_predict_list = [r.split('\n')[-2].split(' ')[-1] for r in response]
        products_predict_list = [r.split('\n')[-1].split(' ')[-1] for r in response]
    except Exception as e:
        return response, [], [], 0, []

    # print('------------------------------------------')
    # print(products_predict_list)

    final_predict = max(set(products_predict_list), key = products_predict_list.count)
    num_different_answer = len(set(products_predict_list))

    # print('------------------------------------------')
    # print(final_predict)
    # print('-'*20)
    # print(num_different_answer)

    return response, products_predict_list, final_predict, num_different_answer, Reagents_predict_list

In [249]:
response, products_predict_list, final_predict, num_different_answer, Reagents_predict_list = predict_with_seperate2(
    target='CC(C)(C(=O)Nc1nncs1)C(c1ccccc1)c1ccc2[nH]ncc2c1.Ic1ccccc1.O=P([O-])([O-])[O-].[K+].[K+].[K+]',
    examples_n=5)

------------------------------------------
You are an expert chemist. You are expected to predict reaction products based on Reactants+Reagents only, with first considering which molecular(s) in Reactants+Reagents are reagents.
     Moleculars are splitted by . 
     Reagents either (i) have none of their constituent atoms in the products, or (ii) appear completely unchanged in the product SMILES. 
 Focus only on the given starting materials and conditions. Do not infer additional reagents or steps. Strive for the simplest and most reasonable organic products. Avoid exotic or improbable outcomes. Balance atom counts properly and produce chemically valid SMILES. Do not include yields, mechanisms, or explanatory text. Only output the major product SMILES. 
 You should output two things, Possible Reagents as well as the major product SMILES.
Reactants+Reagents: Ic1n[nH]c2ccc(NCc3ccccc3)cc12.Nc1cccc(B(O)O)c1
Reagents: 
Products: Nc1cccc(-c2n[nH]c3ccc(NCc4ccccc4)cc23)c1

Reactants+Reagents:

In [252]:
final_predict

'CC(C)(C(=O)Nc1nncs1)C(c1ccccc1)c1ccc2[nH]ncc2c1'

In [250]:
response

['[K+].[K+].[K+]\nProducts: CC(C)(C(=O)Nc1nncs1)C(c1ccccc1)c1ccc2[nH]ncc2c1',
 '[K+].[K+].[K+]\nProducts: CC(C)(C(=O)Nc1nncs1)C(c1ccccc1)c1ccc2[nH]ncc2c1',
 '[K+].[K+].[K+]\nProducts: CC(C)(C(=O)Nc1nncs1)C(c1ccccc1)c1ccc2[nH]ncc2c1',
 '[K+].[K+].[K+]\nProducts: CC(C)(C(=O)Nc1nncs1)C(c1ccccc1)c1ccc2[nH]ncc2c1.Ic1ccccc1.O=P([O-])([O-])[O-]',
 '[K+].[K+].[K+]\nProducts: CC(C)(C(=O)Nc1nncs1)C(c1ccccc1)c1ccc2[nH]ncc2c1']

evaluation

In [296]:
def experiment_eval_separate(test_number = 10, model = "gpt35", cot_type = "simi", examples_n=20, samples_n=5, tempreture=1,
                                 test = separate_data_test, train=separate_data_train, separate_method = 1):
    # query
    correct = 0
    valid_count = 0
    correct_top_1 = 0
    correct_reaction_type_rate_list = []

    test_ = test.sample(test_number, random_state=42) # TODO
    for idx, row in tqdm(test_.iterrows()):
        reactant = row['reactants_smiles']
        product = row['products_smiles']
        # index = row['index']
        
        print('begin!',idx)

        if separate_method == 1:
            response, products_predict_list, final_predict, num_different_answer = predict_with_seperate1(reactant, product,
                                                                                                                                   model = model, cot_type = cot_type, 
                                                                                                          examples_n=examples_n, samples_n=samples_n,
                                                                                                          temperature=tempreture,
                                                                                                          train=train)
        elif separate_method == 2:
            response, products_predict_list, final_predict, num_different_answer, Reagents_predict_list = predict_with_seperate2(reactant,
                                                                                                                                   model = model, cot_type = cot_type, 
                                                                                                          examples_n=examples_n, samples_n=samples_n,
                                                                                                          temperature=tempreture,
                                                                                                          train=train)
        if products_predict_list == []: # invalid answer
            continue

        correct_flag = False
        
        for pred in products_predict_list:
            try:
                mol = Chem.MolFromSmiles(pred)
                pred = Chem.MolToSmiles(mol)
            except Exception as e:
                continue
            
            valid_count += 1
            pred_list = pred.split(".")
            if product in pred_list:
                correct_flag = True
                break
        
        if final_predict == product:
            correct_top_1 += 1
        if correct_flag == True:
            correct += 1

        # correct_reaction_type = 0 
        # for r in predicted_reaction_type:
        #     if r == reaction_type:
        #         correct_reaction_type += 1
        # correct_reaction_type_rate = correct_reaction_type/len(predicted_reaction_type)
        # correct_reaction_type_rate_list.append(correct_reaction_type_rate)

    top_1_acc = correct_top_1/len(test_)
    top_n_acc = correct/len(test_)
    valid_rate = valid_count/(len(test_)*samples_n)
    # mean_correct_reaction_type_rate = np.mean(correct_reaction_type_rate_list)
    return top_1_acc, top_n_acc, valid_rate


In [300]:
# 用USPTO作为训练集，USPTO作为测试集
log = pd.DataFrame(columns=['dataset_name', 'model', 'cot_type', 'test_number','examples_n', 'samples_n', 
                             'temperature',
                             'top_1_acc', 'top_n_acc', 'valid_rate'])

model = ["davinci"]
cot_type = ['simi']
examples_n = [20]
tempreture = [0.5]
samples_n = [5]
sm_list = [1]

dataset_name = "usptosep_usptosep"

for m in model:
    for c in cot_type:  
        for t in tempreture:
            for sm in sm_list:
                top_1_acc, top_n_acc, valid_rate = experiment_eval_separate(test_number = 20, model = m, cot_type = c, 
                                                                            examples_n = 20, samples_n=5, tempreture=t,
                                                                            separate_method = sm)
                                                                            # train = separate_data, test = separate_data)
                log.loc[len(log)] = [dataset_name, m, c, 20, 20, 5, t, top_1_acc, top_n_acc, valid_rate]
                log.to_csv(f'log/separate/[sepa{sm}]{dataset_name}_{m}_{c}_{t}.csv', index=False)


0it [00:00, ?it/s]

begin! 471858


1it [00:06,  6.43s/it]

------------------------------------------
You are an expert chemist. Given only the reactants and reagents (some examples might not have reagents), predict the major organic reaction products as valid SMILES, following these principle.
     Moleculars are splitted by .  
 Focus only on the given starting materials and conditions. Do not infer additional reagents or steps. Strive for the simplest and most reasonable organic products. Avoid exotic or improbable outcomes. Balance atom counts properly and produce chemically valid SMILES. Do not include yields, mechanisms, or explanatory text. Only output the major product SMILES. 
 Catalysts i.e. Reagents either (i) have none of their constituent atoms in the products, or (ii) appear completely unchanged in the product SMILES.
Reactants: CCOc1cc(C(CC(=O)O)N2C(=O)c3ccc([N+](=O)[O-])cc3C2=O)ccc1OC.NOCc1ccccc1
Reagents: C1CCOC1.Cl.O=C(c1ncc[nH]1)c1ncc[nH]1
Products: CCOc1cc(C(CC(=O)NOCc2ccccc2)N2C(=O)c3ccc([N+](=O)[O-])cc3C2=O)ccc1OC

Reacta

[23:02:01] Explicit valence for atom # 12 O, 3, is greater than permitted
2it [00:11,  5.72s/it]

...

------------------------------------------
You are an expert chemist. Given only the reactants and reagents (some examples might not have reagents), predict the major organic reaction products as valid SMILES, following these principle.
     Moleculars are splitted by .  
 Focus only on the given starting materials and conditions. Do not infer additional reagents or steps. Strive for the simplest and most reasonable organic products. Avoid exotic or improbable outcomes. Balance atom counts properly and produce chemically valid SMILES. Do not include yields, mechanisms, or explanatory text. Only output the major product SMILES. 
 Catalysts i.e. Reagents either (i) have none of their constituent atoms in the products, or (ii) appear completely unchanged in the product SMILES.
Reactants: CC(C)CC(N)CN1CCCC1C(=O)O.O=C(Cl)C12CC3CC(CC(C3)C1)C2.[NH-]CCCCc1ccccc1
Reagents: CCN(CC)CC.ClCCl
Products: CC(C)CC(CN1CCCC1C(=O)O)NC(=O)C12CC3CC(CC(C3)C1)C2

Reactants: C1NC2CC3CC1CC(C3)C2.O=C(Cl)c1ccc([N

19it [01:33,  4.95s/it]

------------------------------------------
You are an expert chemist. Given only the reactants and reagents (some examples might not have reagents), predict the major organic reaction products as valid SMILES, following these principle.
     Moleculars are splitted by .  
 Focus only on the given starting materials and conditions. Do not infer additional reagents or steps. Strive for the simplest and most reasonable organic products. Avoid exotic or improbable outcomes. Balance atom counts properly and produce chemically valid SMILES. Do not include yields, mechanisms, or explanatory text. Only output the major product SMILES. 
 Catalysts i.e. Reagents either (i) have none of their constituent atoms in the products, or (ii) appear completely unchanged in the product SMILES.
Reactants: CCOC(=O)C#CC1CC1.COCOc1ccc(I)cc1.OB(O)c1ccccc1
Reagents: CN(C)C=O.Cl[Pd]Cl.N#Cc1ccccc1.N#Cc1ccccc1.O=C([O-])[O-].[K+].[K+]
Products: CCOC(=O)C(=C(c1ccc(OCOC)cc1)C1CC1)c1ccccc1

Reactants: C1=CCNC1.NC(=O)O

20it [01:37,  4.87s/it]

------------------------------------------
You are an expert chemist. Given only the reactants and reagents (some examples might not have reagents), predict the major organic reaction products as valid SMILES, following these principle.
     Moleculars are splitted by .  
 Focus only on the given starting materials and conditions. Do not infer additional reagents or steps. Strive for the simplest and most reasonable organic products. Avoid exotic or improbable outcomes. Balance atom counts properly and produce chemically valid SMILES. Do not include yields, mechanisms, or explanatory text. Only output the major product SMILES. 
 Catalysts i.e. Reagents either (i) have none of their constituent atoms in the products, or (ii) appear completely unchanged in the product SMILES.
Reactants: CC1(C)CCC(O)c2c(CO[Si](C)(C)C(C)(C)C)cccc2C1
Reagents: CC#N.CCN(CC)CC.CS(=O)(=O)Cl.O.[Cl-].[Li+]
Products: CC1(C)CC=Cc2c(CO[Si](C)(C)C(C)(C)C)cccc2C1

Reactants: CCS(=O)(=O)NC1CCCCc2ccc(F)cc21.CI
Reagents

In [272]:
# # 用USPTO作为训练集，5reaction作为测试集
# log = pd.DataFrame(columns=['dataset_name', 'model', 'cot_type', 'test_number','examples_n', 'samples_n', 
#                              'temperature',
#                              'top_1_acc', 'top_n_acc', 'valid_rate'])

# model = ["davinci"]
# cot_type = ['simi']
# examples_n = [20]
# tempreture = [0.5]
# samples_n = [5]
# sm_list = [1]

# dataset_name = "usptosep_5reaction"

# for m in model:
#     for c in cot_type:  
#         for t in tempreture:
#             for sm in sm_list:
#                 top_1_acc, top_n_acc, valid_rate = experiment_eval_separate(test_number = 20, model = m, cot_type = c, 
#                                                                             examples_n = 20, samples_n=5, tempreture=t,
#                                                                             separate_method = sm,
#                                                                             train = separate_data, test = separate_reagents_5reaction)
#                 log.loc[len(log)] = [dataset_name, m, c, 20, 20, 5, t, top_1_acc, top_n_acc, valid_rate]
#                 log.to_csv(f'log/separate/[sepa{sm}]{dataset_name}_{m}_{c}_{t}.csv', index=False)


0it [00:00, ?it/s]

begin! 7458


1it [00:05,  5.70s/it]

------------------------------------------
You are an expert chemist. Given only the reactants and reagents (some examples might not have reagents), predict the major organic reaction products as valid SMILES, following these principle.
     Moleculars are splitted by .  
 Focus only on the given starting materials and conditions. Do not infer additional reagents or steps. Strive for the simplest and most reasonable organic products. Avoid exotic or improbable outcomes. Balance atom counts properly and produce chemically valid SMILES. Do not include yields, mechanisms, or explanatory text. Only output the major product SMILES. 
 Catalysts i.e. Reagents either (i) have none of their constituent atoms in the products, or (ii) appear completely unchanged in the product SMILES.
Reactants: CC(C)(C)OC(=O)c1ccc(B2OC(C)(C)C(C)(C)O2)cc1F.Nc1nccnc1Cl
Reagents: 
Products: CC(C)(C)OC(=O)c1ccc(-c2nccnc2N)cc1F

Reactants: Cc1c(C(=O)Nc2ccc(B3OC(C)(C)C(C)(C)O3)cc2)c(=O)n(-c2ccccc2)n1C.Nc1ncc(Br)nc1Br


2it [00:17,  9.04s/it]

...

------------------------------------------
You are an expert chemist. Given only the reactants and reagents (some examples might not have reagents), predict the major organic reaction products as valid SMILES, following these principle.
     Moleculars are splitted by .  
 Focus only on the given starting materials and conditions. Do not infer additional reagents or steps. Strive for the simplest and most reasonable organic products. Avoid exotic or improbable outcomes. Balance atom counts properly and produce chemically valid SMILES. Do not include yields, mechanisms, or explanatory text. Only output the major product SMILES. 
 Catalysts i.e. Reagents either (i) have none of their constituent atoms in the products, or (ii) appear completely unchanged in the product SMILES.
Reactants: O=C(O)c1ccc(C(F)(F)F)cc1
Reagents: COC(=O)Cc1c(N(C)C)nc(Cc2ccc(N)cc2)nc1N(C)C
Products: COC(=O)Cc1c(N(C)C)nc(Cc2ccc(NC(=O)c3ccc(C(F)(F)F)cc3)cc2)nc1N(C)C

Reactants: Cc1cc(N=C=O)ccc1Oc1ncc(Br)cn1
Reagents

[08:20:05] Can't kekulize mol.  Unkekulized atoms: 17 18 19 20 21 22 23
[08:20:05] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9 11 12 13 21
[08:20:05] Can't kekulize mol.  Unkekulized atoms: 17 18 19 20 21 22 23
[08:20:05] SMILES Parse Error: unclosed ring for input: 'Cc1cc(Oc2nccc3occc(-c4cccc2Br)c4)ccc1B1OC(C)(C)C(C)(C)O1'
19it [02:33,  4.81s/it]

------------------------------------------
You are an expert chemist. Given only the reactants and reagents (some examples might not have reagents), predict the major organic reaction products as valid SMILES, following these principle.
     Moleculars are splitted by .  
 Focus only on the given starting materials and conditions. Do not infer additional reagents or steps. Strive for the simplest and most reasonable organic products. Avoid exotic or improbable outcomes. Balance atom counts properly and produce chemically valid SMILES. Do not include yields, mechanisms, or explanatory text. Only output the major product SMILES. 
 Catalysts i.e. Reagents either (i) have none of their constituent atoms in the products, or (ii) appear completely unchanged in the product SMILES.
Reactants: CC1(C)OB(c2ccc(Oc3ccc(F)cc3)cc2)OC1(C)C.C[C@H](Nc1cc(C(=O)OC(C)(C)C)nc(Cl)n1)C(N)=O
Reagents: 
Products: C[C@H](Nc1cc(C(=O)OC(C)(C)C)nc(-c2ccc(Oc3ccc(F)cc3)cc2)n1)C(N)=O

Reactants: CC1(C)COB(c2ccc(F)c(-c

20it [02:37,  7.87s/it]

------------------------------------------
You are an expert chemist. Given only the reactants and reagents (some examples might not have reagents), predict the major organic reaction products as valid SMILES, following these principle.
     Moleculars are splitted by .  
 Focus only on the given starting materials and conditions. Do not infer additional reagents or steps. Strive for the simplest and most reasonable organic products. Avoid exotic or improbable outcomes. Balance atom counts properly and produce chemically valid SMILES. Do not include yields, mechanisms, or explanatory text. Only output the major product SMILES. 
 Catalysts i.e. Reagents either (i) have none of their constituent atoms in the products, or (ii) appear completely unchanged in the product SMILES.
Reactants: BrCC1CC1
Reagents: OCCOc1ccccc1O
Products: OCCOc1ccccc1OCC1CC1

Reactants: O=[N+]([O-])c1ccccc1F
Reagents: NC1CC1
Products: O=[N+]([O-])c1ccccc1NC1CC1

Reactants: O=C(Cl)C1CC1
Reagents: CCC(C)Oc1ccc(OCCN)

#### ONLY CREATE PROMPTS

In [ ]:
def get_prompts_with_seperate1(target, target_product, cot_type = "simi", examples_n=20, train=separate_reagents_5reaction):
    '''
    parameters:
        examples_n: shot number
        samples_n: sample number
        cot_type: choose demonstration type
    examples:
        Reactants: {example[0]}\nReagents: {example[1]}\nProducts: {example[2]}\n
    predicts:
        Products only
    '''
    
    assert cot_type in ["simi", "random"]

    if cot_type == "simi":
        # get similar molecules
        simi = top_n_scaffold_similar_molecules(target,list(train['scaffold_fp']), list(train['reactants_smiles']), n=examples_n)
        examples = []
        for i in range(examples_n):
            examples.append((train.iloc[simi[1][i]]['reactants_smiles_only'], train.iloc[simi[1][i]]['Reagents_smiles'], train.iloc[simi[1][i]]['products_smiles']))

    elif cot_type == "random":
        examples = []
        random_idx = np.random.randint(0, len(train), examples_n)
        for i in random_idx:
            examples.append((train.iloc[i]['reactants_smiles_only'], train.iloc[i]['Reagents_smiles'], train.iloc[i]['products_smiles']))

    elif cot_type == "mixed": # TODO
        # 各个种类的反应都有一些
        examples = []

    elif cot_type == "different": # TODO
        # get different molecules
        pass
    
    final_prompt = get_ICL_prompts_separate1(target, target_product, examples)


In [ ]:
def predict_with_seperate2(target, model = "gpt35", cot_type = "simi", examples_n=20, samples_n=5, temperature=0.5, train=separate_reagents_5reaction):
    '''
    parameters:
        examples_n: shot number
        samples_n: sample number
        cot_type: choose demonstration type
    examples:
        Reactants+Reagents: {example[0]}\nReagents: {example[1]}Products: {example[2]}\n
    predicts:
        Reagents + Products
    '''
    
    assert cot_type in ["simi", "random"]

    if cot_type == "simi":
        # get similar molecules
        simi = top_n_scaffold_similar_molecules(target,list(train['scaffold_fp']), list(train['reactants_smiles']), n=examples_n)
        examples = []
        for i in range(examples_n):
            examples.append((train.iloc[simi[1][i]]['reactants_smiles'], train.iloc[simi[1][i]]['Reagents_smiles'], train.iloc[simi[1][i]]['products_smiles']))

    elif cot_type == "random":
        examples = []
        random_idx = np.random.randint(0, len(train), examples_n)
        for i in random_idx:
            examples.append((train.iloc[i]['reactants_smiles'], train.iloc[i]['Reagents_smiles'], train.iloc[i]['products_smiles']))

    elif cot_type == "mixed": # TODO
        # 各个种类的反应都有一些
        examples = []

    elif cot_type == "different": # TODO
        # get different molecules
        pass
    
    final_prompt = get_ICL_prompts_separate2(target, examples)